In [ ]:
%reload_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import matplotlib.pylab as pylab
import pandas as pd
from operator import itemgetter

from warnings import filterwarnings

pylab.rcParams['figure.figsize'] = 5, 5
plt.rc("font", size=10)


plt.rc('xtick', color='k', labelsize='medium', direction='in')
plt.rc('xtick.major', size=8, pad=12)
plt.rc('xtick.minor', size=8, pad=12)

plt.rc('ytick', color='k', labelsize='medium', direction='in')
plt.rc('ytick.major', size=8, pad=12)
plt.rc('ytick.minor', size=8, pad=12)

So far we have fitted our curves and doing so we have found the  best model explaining the point that we had for the fitting.

We also saw that we could choose different models according to how much the improvement obtained was worth the complexification of the model. But again we did it on the whole data that we had. We never really check how well our data was generalizing to points never seen before, or by how much the model we found was subject to outliers.

Machine learning procedures allow us to take those considerations into account. After highlighting the few caveats of the procedures we have used in the former notebook, we will introduce the foundation of the machine learning way to model.

More particularly we will see that the machine learning paradigm modifies the function to optimize that we have seen before by adding a penalty to covariables that generalize badly. We will also see that in a machine learning procedure, the generalization is approached by fitting and evaluating mutliple times your model on subset of your data.

**The machine learning paradigm emphasizes the importance of building a general model that will be good at dealing with future, unknown, data points rather than being the best model on the data that we have now.**


# Table Of Content: <a id="toc"></a>


* [**motivating example**](#motivation)

* [**Linear regression**](#linear)
    * [approach 1: a simple linear regression](#linear-1)
    * [approach 2: adding regularization and validation set](#linear-2)
    * [approach 3 : k-fold cross-validation](#linear-3)
    * [approach 4 : a "classical" ML pipeline](#linear-4)
    
* [**Logistic regression**](#toy-example-lr)
    * [Exercise: Logistic regression to detect breast cancer malignancy](#LR-exercise)

* [**Imbalanced dataset**](#imbalanced)
* [**A few VERY IMPORTANT words on leakage.**](#leakage)

* [**Support Vector Machine**](#svm)
    * [SVM for Classification](#svm-c)
        * [introduction](#formal-svm-c)
        * [Toy example to visualize SVMC.](#toy-example-svm-c)
        * [SVM Classifier pipeline.](#svm-c-pipeline)
    * [SVM for Regression.](#svm-r)
    
* [**Decision tree modeling .**](#decision-tree)
    * [Simple decision tree for classification.](#simple-tree-c)
        * [Toy example to visualize decision tree.](#toy-decision-tree)
        * [Single decision tree pipeline.](#single-tree-pipeline)
    * [Random Forest in classification](#rf-c)
        * [Exercise: Random Forest on the breast cancer dataset](#rf-exo)
        * [RF annex 1: too many features](#rf-a1)
    * [Random Forest in regression](#rf-r)
    
* [**Conclusion**](#conclusion)


* [**Classification exercise**](#exo-classif)

* [**Regression Exercise**](#exo-regression)
* [**Annexes**](#annex)

There it seems that we fare much better in the cubic scenario.

# motivating example <a id="motivation"></a>

[Acharjee et al.2016](https://bmcbioinformatics.biomedcentral.com/articles/10.1186/s12859-016-1043-4) propose several -omic dataset which they used to predict and gain knowledge on various phenotypic traits in potatos.

Here, we will concentrate on the their transcriptomics dataset and the phenotypic trait of the potato coloration.

We have pre-selected and normalized the 200 most promising genes (out of ~15 000).

In [ ]:
file_metadata = "data/potato_data.phenotypic.csv"
file_data = "data/potato_data.transcriptomic.top200norm.csv"

df = pd.read_csv( file_metadata , index_col=0 )
dfTT = pd.read_csv( file_data , index_col=0)

df.shape

For the sake of our story, we will imagine that out of the 86 potatos in the data, we have only 73 at the time of our experiment.

We put aside the rest for later.

In [ ]:
i1 = df.index[:73]
i2 = df.index[73:]

In [ ]:
X = dfTT.loc[i1 , :]
X.head()

In [ ]:
y = df.loc[i1 , "Flesh Colour"]
y.describe()

In [ ]:
sns.histplot(y)

[Back to ToC](#toc)

# Linear regression  <a class="anchor" id="linear"></a>

## approach 1: a simple linear regression <a class="anchor" id="linear-1"></a>

Let's fit a simple linear model with our gene expression values, and see what happens


In [ ]:
import statsmodels.api as sm
## importing scoring functions
from sklearn.metrics import r2_score , mean_squared_error

Xc = sm.add_constant(X)## adding the intercept to the model

model = sm.OLS( y , Xc ) 
model = model.fit() 

# predict
y_pred = model.predict( Xc )

# evaluate the prediction
print(f"R-squared score: { r2_score( y , y_pred ) :.2f}")
print(f"mean squared error: { mean_squared_error( y , y_pred ) :.2f}")


**Wow!!** this is a perfect fit.

But if you know anything about biology, or data analysis, then you likely suspect something wrong is happening.


Indeed, at the moment, our claim is that our model can predict flesh color perfectly (RMSE=0.0) from the normalized expression of these 200 genes.

But, say we now have some colleagues who come to us with some new potato data:


In [ ]:

## now we use the leftover data points:
Xnew = dfTT.loc[i2 , :]
ynew = df.loc[i2 , "Flesh Colour"]

## apply the model on the new data
ynew_pred = model.predict( sm.add_constant(Xnew) )

# evaluate the prediction
print(f"new data R-squared score: { r2_score( ynew , ynew_pred ) :.2f}")
print(f"new data mean squared error: { mean_squared_error( ynew , ynew_pred ) :.2f}")

In [ ]:
plt.scatter( y , y_pred , label = 'training data' )
plt.scatter( ynew , ynew_pred , label = 'new data' )
plt.xlabel('observed values')
plt.ylabel('predicted values')
plt.legend()

As expected, the performance on the new data is not as good as with the data we used to train the model.

We have **overfitted** the data.

But, before we process with how we can try to avoid these problems,
 we are going to switch to `scikit-learn` objects. This will make our life easier down the line.
 
---
 
Doing the thing we just did with `scikit-learn`:

In [ ]:
## we import elements from sklearn
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score , mean_squared_error


# create the regression object
lin_reg = LinearRegression()

# fit it with our data
lin_reg.fit(X,y)

# predict
y_pred = lin_reg.predict( X )

# evaluate the prediction
print(f"R-squared score: { r2_score( y , y_pred ) :.2f}")
print(f"mean squared error: { mean_squared_error( y , y_pred ) :.2f}")

In [ ]:
## now we use the leftover data points:
Xnew = dfTT.loc[i2 , :]
ynew = df.loc[i2 , "Flesh Colour"]

## apply the model on the new data
ynew_pred = lin_reg.predict( Xnew )

# evaluate the prediction
print(f"new data R-squared score: { r2_score( ynew , ynew_pred ) :.2f}")
print(f"new data mean squared error: { mean_squared_error( ynew , ynew_pred ) :.2f}")

In [ ]:
plt.scatter( y , y_pred , label = 'training data' )
plt.scatter( ynew , ynew_pred , label = 'new data' )
plt.xlabel('observed values')
plt.ylabel('predicted values')
plt.legend()

> NB: the models are not perfectly equivalent here. This is because (a) statsmodels does some normalization under the hood which sklearn doesn't (b) there are several ways to overfit this data and the two library use two different ones.

### ANNEX : "generic" sklearn usage 

The main library we will be using for machine learning is scikit-learn.

It should go without saying that if you have any questions regarding its usage and capabilities, your first stop should be their [website](https://scikit-learn.org/stable/),
especially since it provides plenty of [examples](https://scikit-learn.org/stable/auto_examples/ensemble/plot_voting_decision_regions.html#sphx-glr-auto-examples-ensemble-plot-voting-decision-regions-py), [guides](https://scikit-learn.org/stable/user_guide.html), and [tutorials](https://scikit-learn.org/stable/tutorial/index.html#tutorial-menu).

Nevertheless, we introduce here the most common behavior of sklearn object.

Indeed, sklearn implement machine learning algorithms (random forest, clustering algorithm,...), as well as all kinds of preprocessers (scalin, missing value imputation,...) with a fairly consistent interface.

Most methods must first be instanciated as an object from a specific class:

```python
## import the class, here RandomForestClassifier
from sklearn.ensemble import RandomForestClassifier

## instanciate the class object:
my_clf = RandomForestClassifier()
```

As it stands, the object is just a "naive" version of the algorithm.

The next step is then to feed the object data, so it can learn from it. This is done with the `.fit` method:

```python
my_clf.fit( X , y )
```
> In this context, `X` is the data and `y` is the objective to attain. When the object is not an ML algorithm but a preprocessor, you only give the `X`

Now that the object has been trained with your data, you can use it. For instance, to:
* `.transform` your data (typically in the case of a preprocessor)
* `.predict` some output from data (typically in the case of an ML algorithm, like a classifier)

```python
y_predicted = clf.predict(X)  # predict classes of the training data

## OR 

X_scaled = myScaler.transform(X)  # apply a transformation to the data
```

Last but not least, it is common in example code to "fit and transform" a preprocesser in the same line using `.fit_transform`

```python
X_scaled = myNaiveScaler.fit_transform(X)  # equivalent to myNaiveScaler.fit(X).transform(X)
```

That's the basics. You will be able to experiment at length with this and go well beyond it.

<br>

---

Anyhow, here we could still use the model that we have created, 
but we would agree that reporting the perfect performance we had with our training data would be misleading.

To honestly report the performance of our model, we measure it on a **set of data that has not been used at all to train it: the *test set*.**



To that end, we typically begin by dividing our data into :

 * **train** set : find the best model
 * **test** set  : give an honest evaluation of how the model perform on completely new data.

![train_test](image/train_test.png)

In [ ]:
X_test = Xnew
y_test = ynew

[Back to ToC](#toc)

## approach 2: adding regularization and validation set <a class="anchor" id="linear-2"></a>

In the case of a Least Square fit, the function you are minimizing looks like:

$\sum_i (y_i-f(\pmb X_i,\pmb{\beta}))^2$

, so the sum of squared difference between the observation and the predictions of your model.


**Regularization** is a way to reduce overfitting, and in the case of the linear model
we do so by adding to this function a **penalization term which depends on coefficient weights**.

In brief, the stronger the coefficient, the higher the penalization. So only coefficients which bring more fit than penalization will be kept.


> Note : we report here the formulas used in `scikit-learn` functions. Other libraries may have a different parameterization, but the concepts stay the same

$\frac{1}{2n}\sum_i (y_i-f(\pmb X_i,\pmb{\beta}))^2 + \alpha\sum_{j}|\beta_{j}|$ , **l1 regularization** (Lasso) $\alpha$ being the weight that you put on that regularization 

$\sum_i (y_i-f(\pmb X_i,\pmb{\beta}))^2 + \alpha\sum_{j}\beta_{j}^{2}$ , **l2 regularization** (Ridge) 

$\frac{1}{2n}\sum_i (y_i-f(\pmb X_i,\pmb{\beta}))^2 + \alpha\sum_{j}(\rho|\beta_{j}|+\frac{(1-\rho)}{2}\beta_{j}^{2})$ , **elasticnet**


For a deeper understanding of those notions, you may look at :

 * https://www.datacamp.com/community/tutorials/tutorial-ridge-lasso-elastic-net

 * https://towardsdatascience.com/regularization-in-machine-learning-76441ddcf99a



> NB: Regularization generalize to maximum likelihood contexts as well)




Let's try that on our data:

In [ ]:
%%time
from sklearn.linear_model import SGDRegressor


logalphas = []

coef_dict = {'name' : [],
             'coefficient' : [],
             'log-alpha' : []}
r2 = []

for alpha in np.logspace(-2,2,50):

    reg = SGDRegressor( penalty='l1' , alpha = alpha )
    reg.fit( X , y )
    
    logalphas.append(np.log10(alpha))
    r2.append( r2_score( y , reg.predict(X) ) )
    
    coef_dict['name'] += list( X.columns )
    coef_dict['coefficient'] += list( reg.coef_ )
    coef_dict['log-alpha'] += [np.log10(alpha)]* len(X.columns )

coef_df = pd.DataFrame(coef_dict)

In [ ]:

fig,ax = plt.subplots(1,2,figsize = (14,7))

ax[0].plot(logalphas , r2)
ax[0].set_xlabel("log10( alpha )")
ax[0].set_ylabel("R2")

sns.lineplot( x = 'log-alpha' , y='coefficient' , hue = 'name' , data= coef_df , ax = ax[1] ,legend = False)

fig.suptitle("regression of potato data with an L1 regularization.")



**Micro-exercise:** adapt the code above to generate this plot with an l2 penalty. How do you interpret the difference?


In [ ]:
# %load solutions/solution_03_mini1.py

This is great, but how do we choose which level of regularization we want ?

It is a general rule that **as you decrease $\alpha$, the $R^2$ on the data used for the fit increase**, i.e. you risk overfitting.

Consequently, we cannot choose the value of $\alpha$ parameter from the data used to fit alone; we call such a parameter an **hyper-parameter**.

**Question:** what are other hyper-parameters we could optimize at this point?

---

<br>

In order to find the optimal value of an hyper-parameter, we can separate our training data into:
 * a **train set** : used to fit the model
 * a **validation set** : used to evaluate how our model perform on new data 


In [ ]:
X.shape

In [ ]:
%%time
## adding a validation set

# we will use 60 points to train the model
# and we will use the rest to evaluate the model 
I = list( range(X.shape[0]))
np.random.shuffle( I ) 

I_train = I[:60]
I_valid = I[60:]

X_train = X.iloc[ I_train , : ] 
y_train = y.iloc[ I_train ]

# we will use the rest to evaluate the model
X_valid = X.iloc[ I_valid , : ] 
y_valid = y.iloc[ I_valid ]


logalphas = []

r2_train = []
r2_valid = []

for alpha in np.logspace(-3,2,200):

    reg = SGDRegressor( penalty='l1' , alpha = alpha  )
    reg.fit( X_train , y_train )
    
    logalphas.append(np.log10(alpha))
    r2_train.append( r2_score( y_train , reg.predict(X_train) ) )
    r2_valid.append( r2_score( y_valid , reg.predict(X_valid) ) )
    
## plotting and reporting 
bestI = np.argmax(r2_valid)
bestLogAlpha = logalphas[bestI]
bestR2_valid = r2_valid[bestI]

fig,ax = plt.subplots(figsize = (10,5))
fig.suptitle("best alpha : {:.2f} - validation R2 : {:.2f}".format(10**bestLogAlpha , bestR2_valid))
ax.plot( logalphas, r2_train , label='train set' )
ax.plot( logalphas, r2_valid , label='validation set' )
ax.scatter( [bestLogAlpha] , [bestR2_valid]  , c='red')
ax.set_xlabel("log10( alpha )")
ax.set_ylabel("R2")
ax.set_ylim(-0.1,1.1)
ax.legend()


So now, with the help of a validation set, we can clearly see the phases :
 * **underfitting** : for high $\alpha$, the performance is low for both the train and the validation set
 * **overfitting** : for low $\alpha$, the performance is high for the train set, and low for the validation set
 
We want the equilibrium point between the two where performance is ideal for the validation set.

**Problem :** if you run the code above several time, you will see that the optimal point varies due to the random assignation to train or validation set. 

There exists a myriad of possible strategies to deal with that problem, such as repeating the above many times and taking the average of the results for instance.
Note also that this problem gets less important as the validation set size increases.

<br>

---

<br>

Anyhow, on top of our earlier regression model, we have added :

 * an **hyper-parameter** : $\alpha$, the strength of the regularization term
 * a **validation strategy** for our model in order to avoid overfitting

<br>

That's it, we are now in the world of Machine Learning.

But before we go any further, let's see how this modified model performs on the test data:

In [ ]:

reg = SGDRegressor( penalty='l1' , alpha = 10**bestLogAlpha  )
reg.fit( X , y )

y_pred = reg.predict( X )
print(f"train data R-squared score: { r2_score( y , y_pred ) :.2f}")
print(f"train data mean squared error: { mean_squared_error(  y , y_pred ) :.2f}")


y_test_pred = reg.predict( X_test )

print(f" test data R-squared score: { r2_score( y_test , y_test_pred ) :.2f}")
print(f" test data mean squared error: { mean_squared_error(  y_test , y_test_pred ) :.2f}")


plt.scatter( y , y_pred , label = 'training data' )
plt.scatter( y_test , y_test_pred , label = 'new data' )
plt.xlabel('observed values')
plt.ylabel('predicted values')
plt.legend()

Two things to observe:
 * we still see better performance on the train data than on the test data (generally always the case)
 * the performance on the test set has improved: our model is less overfit and more generalizable

[Back to ToC](#toc)

##  approach 3 : k-fold cross-validation <a class="anchor" id="linear-3"></a>

In the previous approach, we have split our training data into a train set and a validation set.

This approach works well if you have enough data for your validation set to be representative.

Often, we unfortunately do not have enough data for this.

Indeed, we have seen that if we run the code above several time, we see that the optimal point varies due to the random assignation to train or validation set. 


**K-fold cross validation** is one of the most common strategy to try to mitigate this randomness with a limited amount of data.

![k-fold validation](images/kfold.png)

In k-fold cross-validation, you split you data in $k$ subpart, called fold.

Then, for a given hyper-parameter values combination, you actually train $k$ model: each time you use a different fold for validation (and the remaining $k-1$ folds for training).

You then compute the average performance across all fold : this is the **cross-validated performance**.

--- 

We are going to do a simple k-fold manually once, to explore a bit how it works, but in practice you will discover that it is mostly automatized with some of scikit-learn's recipes and objects.

In [ ]:
## Kfold
from sklearn.model_selection import KFold



kf = KFold(n_splits=5 , shuffle=True , random_state=734)
for i, (train_index, valid_index) in enumerate(kf.split(X)):
    print(f"Fold {i}:")
    print(f"  Train: index={train_index}")
    print(f"  Test:  index={valid_index}")


In [ ]:
%%time
logalphas = np.linspace(-2,1,200)

kf = KFold(n_splits=5 , shuffle=True , random_state=6581) ## try changing the random state

fold_r2s = [ [] for i in range(kf.n_splits) ] ## for each fold
cross_validated_r2 = [] # average across folds
   
for j,alpha in enumerate( 10**logalphas ) :

    cross_validated_r2.append(0)
    
    for i, (train_index, valid_index) in enumerate(kf.split(X)):

        ## split train and validation sets
        X_train = X.iloc[ train_index , : ]
        X_valid = X.iloc[ valid_index , : ]

        y_train = y.iloc[ train_index ]
        y_valid = y.iloc[ valid_index ]

        ## fit model for that fold
        reg = SGDRegressor( penalty='l1' , alpha = alpha  )
        reg.fit( X_train , y_train )

        ## evaluate for that fold
        fold_score = r2_score( y_valid , reg.predict(X_valid) )
        
        ## keeping in the curve specific to this fold
        fold_r2s[i].append( fold_score )
        
        ## keeping a tally of the average across folds
        cross_validated_r2[-1] += fold_score/kf.n_splits

        
        

bestI = np.argmax(cross_validated_r2)
bestLogAlpha = logalphas[bestI] 
bestR2_valid = cross_validated_r2[bestI]


## plotting
fig,ax = plt.subplots(figsize = (10,5))

ax.plot( logalphas, cross_validated_r2 , label='cross-validated r2' )
ax.scatter( [bestLogAlpha] , [bestR2_valid]  , c='red')

for i,scores in enumerate(fold_r2s):
    ax.plot( logalphas , scores , label = f'fold {i}' , linestyle='dotted' )

ax.set_xlabel("log10( alpha )")
ax.set_ylabel("R2")
ax.set_ylim(-0.1,1.1)
ax.legend()

fig.suptitle("best alpha : {:.2f} - cross-validated R2 : {:.2f}".format(10**bestLogAlpha , bestR2_valid))



**micro-exercise**: re-fit a model with the alpha we found and check the performance with the *test* data

In [ ]:
# %load solutions/solution_03_mini2.py

There, you can realize that now, for each possible value of our hyper-parameter we fit and evaluate not 1, but $k$ models, here 4.

So, for 200 values of $\alpha$, that means 200x5 = 1000 models to fit and evaluate.

Now, consider that we have other hyper-parameters, such as the type of regularization (L1 or L2),
or how we perform scaling, or whether we consider interactions, and now you understand why Machine Learning can quickly become  computationnaly intensive. 

---


[Back to ToC](#toc)

## approach 4 : a "classical" ML pipeline <a class="anchor" id="linear-4"></a>

We will start back from scratch to recapitulate what we've seen and use scikit-learn to solve the potato problem.


In [ ]:
## full dataset
X = dfTT
y = df[ "Flesh Colour"]


We start by splitting our data in a train and a test set

In [ ]:
from sklearn.model_selection import train_test_split

X_train , X_test , y_train, y_test = train_test_split(X,y , test_size=0.2 )

print('train set size:',len(y_train))
print(' test set size:',len(y_test))

Now we train a model while optimizing some hyper-parameters.

On top of what we've done before, I add a scaling phase, and test l1 or l2 penalties.

Scikit-learn's `GridSearchCV` is useful to explore these more "complex" hyper-parameter spaces.


In [ ]:
%%time
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV



## our pipeline will have 2 consecutive steps
##   * standard scaling : set mean of each feature at 0 and (optionally) standard dev at 1 
##   * linear regression with some regularization
pip_reg = Pipeline([('scaler',StandardScaler()),
                    ('model',SGDRegressor())])



# define the hyperparameters you want to test
# with the range over which you want it to be tested.
# 
# They are given in a dictionary with the structure:
#      pipelineStep__parameter : [set of values to explore]
#                  ^^
#                  note the double underscore _
grid_values = {'scaler__with_std' : [ True , False ],
               'model__penalty':[ 'l1' , 'l2' ],
               'model__alpha':np.logspace(-2,2,200)}


# Feed the pipeline and set of values to the GridSearchCV with the 
# score over which the decision should be taken (here, R^2).
# and the cross-validation scheme, here the number of fold in a stratified k-fold strategy
grid_reg = GridSearchCV(pip_reg, 
                        param_grid = grid_values, 
                        scoring='r2', 
                        cv = 5,
                        n_jobs=-1)

# When the actual fit happens
#  the gridSearchCV object will go through each hyperparameter value combination
#  and fit + evaluate each fold, and averages the score across each fold.
#
#  It then finds the combination that gave the best score and
#  use it to re-train a model with the whole train data
grid_reg.fit(X_train, y_train)

# get the best cross-validated score 
print(f'Grid best score ({grid_reg.scoring}): {grid_reg.best_score_:.3f}')

# print the best parameters
print('Grid best parameter :')
for k,v in grid_reg.best_params_.items():
    print(' {:>20} : {}'.format(k,v))


In [ ]:
## gridSearch CV fits a new estimator with the best hyperparameter values
reg = grid_reg.best_estimator_

y_pred = reg.predict( X_train )
print(f"train data R-squared score: { r2_score( y_train , y_pred ) :.2f}")
print(f"train data mean squared error: { mean_squared_error(  y_train , y_pred ) :.2f}")


y_test_pred = reg.predict( X_test )

print(f" test data R-squared score: { r2_score( y_test , y_test_pred ) :.2f}")
print(f" test data mean squared error: { mean_squared_error(  y_test , y_test_pred ) :.2f}")



plt.scatter( y_train , y_pred , label = 'training data' )
plt.scatter( y_test , y_test_pred , label = 'new data' )
plt.xlabel('observed values')
plt.ylabel('predicted values')
plt.legend()

One can also access the best model parameter:

In [ ]:
pd.DataFrame(grid_reg.cv_results_)

In [ ]:
grid_reg.best_estimator_

In [ ]:
## coefficient of the linear model
grid_reg.best_estimator_['model'].coef_

[back to ToC](#toc)

# Logistic regression <a class="anchor" id="toy-example-lr"></a>

Let's imagine a simple case with 2 groups, and a single feature:

In [ ]:
X1 = np.concatenate( [ np.random.randn(300) , np.random.randn(300)+4 ])
y = np.array( [0]*300 + [1]*300 )

sns.histplot( x=X1,hue = y )

We will use a logistic regression to model the relationship between the class and the feature.

Remember : **Logistic regression does not model the class directly, but rather model the class probabilities** (through the logit transform)

Let's see how regularization affect the class probabilities found by our model:

In [ ]:
from sklearn.linear_model import LogisticRegression

# do not forget to scale the data
X1_norm = StandardScaler().fit_transform(X1.reshape( X1.shape[0] , 1 ))

fig,ax = plt.subplots( figsize = (10,5) )

ax.scatter( X1_norm , y , c = y )

for alpha in [0.01,0.1,1,10]:
    
    # this implementation does not take alpha but rather C = 1/alpha
    C = 1/alpha
    lr = LogisticRegression( penalty = 'l2' , C = C )
    lr.fit(X1_norm , y)
    
    proba = lr.predict_proba(np.linspace(-2,2,100).reshape(-1, 1))
    ax.plot( np.linspace(-2,2,100) , proba[:,1] , label = 'alpha = {}'.format(alpha) )
ax.legend()

We can see that **when $\alpha$ grows the probabilities evolve more smoothly** ie. we have more regularization.

> However, note that all the curves meet at the same point, corresponding to the 0.5 probability.

This is nice, but **our end-goal is to actually be able to predict the classes**, and not just the probabilities.

Our task is not regression anymore, but rather **classification**.

So here, we do not evaluate the model using $R^2$ or log-likelihood, but a classification metric.

we will discuss a few of these metrics, and we will begin by the most common: **Accuracy**


The Accuracy is the proportion of samples which were correctly classified (as either category).

More mathematically:

$$Accuracy = \frac{TP + TN}{TP+FP+FN+TN}$$

![image/TPFP.png](image/TPFP.png)q
Image credit wikipedia user Sharpr for svg version. original work by kakau in a png. Licensed under the [Creative Commons Attribution-Share Alike 3.0 Unported license](https://creativecommons.org/licenses/by-sa/3.0/deed.en).

* TP : True Positive
* FP : False Positive
* TN : True Negative
* FN : False Negative

So you can see that accuracy forces us to make a choice about the **probability threshold we use predict categories**.

0.5 is a common choice, and the default of the `predict` method:


In [ ]:
from sklearn.metrics import accuracy_score
y_predicted = lr.predict(X1_norm)

print( f"Accuracy with a threshold of 0.5 : {accuracy_score(y,y_predicted):.2f}"  )

pd.crosstab( y , y_predicted , rownames = ['observed'] , colnames = ['predicted'] )

But it can be useful to remember that this is only 1 choice among many:

In [ ]:
y_predicted

In [ ]:
threshold = 0.2
y_predicted = lr.predict_proba(X1_norm)[:,1] > threshold
print( f"Accuracy with a threshold of {threshold} : {accuracy_score(y,y_predicted):.2f}"  )
pd.crosstab( y , y_predicted , rownames = ['observed'] , colnames = ['predicted'] )

**Micro-exercise :** modify the threhold in the code above. 
 * in which direction should the threshold move to limit the number of False Positive ?
 * for which application could that be useful ? 


[Back to ToC](#toc)

## Exercise: Logistic regression to detect breast cancer malignancy <a class="anchor" id="LR-exercise"></a>

Let's build a logistic regression model that will be able to predict if a breast tumor is malignant or not

In [ ]:
from sklearn.datasets import load_breast_cancer
#loading the dataset which is comprised in scikit learn already
data = load_breast_cancer()

## we reduce the features because otherwise this problem is a bit too easy ;-)
m = list( map( lambda x : x.startswith("mean ") , data["feature_names"] ) )


X_cancer=data['data'][:,m]
y_cancer= 1-data['target']

#making it into a dataframe
breast_cancer_df=pd.DataFrame(X_cancer,
    columns=data["feature_names"][m])

breast_cancer_df["target"]=y_cancer

breast_cancer_df.head()

In [ ]:
breast_cancer_df.target.value_counts()

* 357 benign samples
* 212 malignant samples

Here, all these covariables / features are defined on very different scales, for them to be treated fairly in their comparison you need to take that into account by scaling.



**task 1:** split the data into a train and a test dataset

Complete the following code cell:

In [ ]:
#split your data

# stratify is here to make sure that you split keeping the repartition of labels unaffected
X_train_cancer, X_test_cancer, y_train_cancer, y_test_cancer = ...


print("fraction of class malignant in train",sum(y_train_cancer)/len(y_train_cancer))
print("fraction of class malignant in test ",sum(y_test_cancer)/len(y_test_cancer) )
print("fraction of class malignant in full ",sum(y_cancer)/len(y_cancer))

**task 2:** design your pipeline and grid search

Complete the following cell to perform hyper-parameter search with the following specification:

* hyperparameters:
    * scaler : no hyper-parameters for the scaler (ie, we will keep the defaults)
    * logistic regression : test different values for `C` and `penalty` 
* score: 'accuracy'
* cross-validation: use 10 folds


> If you want to test the elasticnet penalty, you will also have to adapt the `solver` parameter (cf. the [LogisticRegression documentation](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html) )


In [ ]:
%%time

pipeline_lr_cancer=Pipeline([('scaler',StandardScaler()),
                             ('model',LogisticRegression(solver = 'liblinear'))])

#define the hyper-parameter space to explore
grid_values = { ... }

#define the GridSearchCV object
grid_cancer = GridSearchCV( ... )

#train your pipeline
grid_cancer.fit( ... )


#get the best cross-validated score 
print(f'Grid best score ({grid_cancer.scoring}): {grid_cancer.best_score_:.3f}')
# print the best parameters
print('Grid best parameter :')
for k,v in grid_cancer.best_params_.items():
    print(' {:>20} : {}'.format(k,v))


Correction:

**task 1:** split the data into a train and a test dataset

In [ ]:
# %load -r -13 solutions/solution_03_LR_cancer.py

**task 2:** design your pipeline and grid search

In [ ]:
# %load -r 14- solutions/solution_03_LR_cancer.py

From there we can explore the model a bit further:

In [ ]:
# we can then access the coefficient of the model, to assess the importance of the different parameters:

w_lr_cancer = grid_cancer.best_estimator_['model'].coef_[0]

sorted_features=sorted( zip( breast_cancer_df.columns , w_lr_cancer  ),
                       key= lambda x : np.abs( x[1] ) , # sort by absolute value
                       reverse=True)

print('Features sorted per importance in discriminative process')
for f,ww in sorted_features:
    print('{:>25}\t{:.3f}'.format(f,ww))

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix

y_cancer_test_pred = grid_cancer.predict(X_test_cancer)

# get the confusion matrix:
confusion_m_cancer = confusion_matrix(y_test_cancer, y_cancer_test_pred)

## recipe to plot the confusion matrix : 
plt.figure(figsize=(5,4))
sns.heatmap(confusion_m_cancer, annot=True)
plt.title(f'Accuracy:{accuracy_score(y_test_cancer,y_cancer_test_pred):.3f}')
plt.ylabel('True label')
plt.xlabel('Predicted label')


So, with its default threshold of 0.5, this model tends to produce more False Positive (ie. benign cancer seen as malignant), than False Negative (ie. malignant cancer seen as benign).

Depending on the particular of the problem we are trying to solve, that may be a desirable outcome.

Whatever the case, it is always interesting to explore a bit more : we will plot how each possible threshold affect the True Positive Rate and the False Positive Rate (**TPR and FPR**) : this is the Receiver Operating Characteristic c urve (**ROC curve**)

In [ ]:
from scipy.special import expit
from sklearn.metrics import roc_curve
from sklearn.metrics import auc

# 1. the predict_proba gives you the probability in each class.
#       we want to have the probability of being of class 1 (malignant), so we take the second column
y_proba_lr_cancer = grid_cancer.predict_proba(X_test_cancer)[:,1]

#   for the logistic regression, that score is the logit, so can convert it back to 
#   probabilities with the expit function (which is the inverse of the logit)
#y_proba_lr_cancer = expit( y_score_lr_cancer )

# 2. this calculates the ROC curve : TPR and FPR for each threshold of score
fpr_lr_cancer, tpr_lr_cancer, threshold_cancer = roc_curve(y_test_cancer, y_proba_lr_cancer)

# we find the point corresponding to a 0.5 theshold
keep = np.argmin( np.abs( threshold_cancer - 0.5 ) )

# we compute the area under the ROC curve
roc_auc_lr_cancer = auc( fpr_lr_cancer, tpr_lr_cancer )

# 3. plotting 
plt.figure()
plt.xlim([-0.01, 1.01])
plt.ylim([-0.01, 1.01])
plt.plot(fpr_lr_cancer, tpr_lr_cancer, lw=3, label='LogRegr ROC curve\n (area = {:0.2f})'.format(roc_auc_lr_cancer))
plt.plot(fpr_lr_cancer[keep], tpr_lr_cancer[keep],'ro',label='threshold=0.5')
plt.xlabel('False Positive Rate', fontsize=16)
plt.ylabel('True Positive Rate', fontsize=16)
plt.title('ROC curve (logistic classifier)', fontsize=16)
plt.legend(loc='lower right', fontsize=13)
plt.plot([0, 1], [0, 1], color='navy', lw=3, linestyle='--')
plt.show()

So with this ROC curve, we can see how the model would behave on different thresholds.

**Question:** we have marked the 0.5 threshold on the plot. Where would a higher threshold be on the curve?

<br>

---

You can see that when plotting the ROC curve, I have also computed its "Area Under the Curve" : 
indeed ROC AUC is another common metric when doing classification.


For now, let's put this aside briefly to explore a very common problem in classification : imbalance

<br>

[Back to the ToC](#toc)

<br>

## Imbalanced dataset <a class="anchor" id="imbalanced"></a> 

Let's use the same small example as before, but now instead of 300 sample of each class, imagine we only have 30 of class 1:

In [ ]:
X1 = np.concatenate( [ np.random.randn(300) , np.random.randn(30)+2 ])
y = np.array( [0]*300 + [1]*30 )

# do not forget to scale the data
X1_norm = StandardScaler().fit_transform(X1.reshape( X1.shape[0] , 1 ))

fig,ax = plt.subplots(1,2, figsize = (14,5) )

sns.histplot( x=X1,hue = y , ax =ax [0])


ax[1].scatter( X1_norm , y , c = y )

for alpha in [0.01,0.1,1,10]:
    
    # this implementation does not take alpham but rather C = 1/alpha
    C = 1/alpha
    lr = LogisticRegression( penalty = 'l2' , C = C )
    lr.fit(X1_norm , y)
    
    proba = lr.predict_proba(np.linspace(-2,3,100).reshape(-1, 1))
    ax[1].plot( np.linspace(-2,3,100) , proba[:,1] , label = 'alpha = {}'.format(alpha) )
ax[1].legend()

You can see that now the point where the probability curves for different alpha converge is not 0.5 anymore...

Also, the probability says fairly low even at the right end of the plot.

In [ ]:
y_predicted = lr.predict(X1_norm)
print( f"Accuracy with a threshold of 0.5 : {accuracy_score(y,y_predicted):.2f}"  )
pd.crosstab( y , y_predicted )

So, most of the class 1 samples are miss-classified (22/30), but we still get a very high accuracy...

That is because, by contruction, both the **logistic regression and accuracy score do not differentiate False Positive and False Negative**.

And the problem gets worse the more imbalance there is :

In [ ]:
from sklearn.metrics import recall_score

## RECALL = TP / (TP + FN)

recall_list = []
acc_list = []

imbalance_list = np.linspace(0,1,50)

alpha = 1
for imbalance in imbalance_list:

    n0 = 300
    n1 = int( n0 * (1 - imbalance) )
    if n1 == 0:
        n1 = 1
    
    X1 = np.concatenate( [ np.random.randn(n0) , np.random.randn(n1)+2 ])
    y = np.array( [0]*n0 + [1]*n1 )

    X1_norm = StandardScaler().fit_transform(X1.reshape( X1.shape[0] , 1 ))
    
    C = 1/alpha
    lr = LogisticRegression( penalty = 'l2' , C = C )
    lr.fit(X1_norm , y)
    
    y_predicted = lr.predict(X1_norm)
    
    recall_list.append( recall_score( y , y_predicted ) )
    acc_list.append( accuracy_score(y,y_predicted) )

        
fig,ax=plt.subplots(figsize = (10,4))
ax.plot( imbalance_list , acc_list , label='accuracy' )
ax.plot( imbalance_list , recall_list , label='recall' )
ax.set_xlabel("imbalance")
ax.legend()

So not only does the precision get worse, the **accuracy actually gets higher as there is more imbalance!**

So the problem here may be 2-fold:
 * imbalance in our dataset skews the **logistic regression** toward a particular outcome
 * **accuracy** is not able to differenciate between False Positive and False Negative, and so it is **blind to imbalance**

Consequently, the solutions will have to come both from the model, and from the metric we are using.


**For the logistic regression**:
 * we will re-weight sample according to their class frequency, so that they are more important during the fitting.
 * in sklearn : `LogisticRegression( ... , class_weight='balanced')`
 
<br> 

**For the metric**, there exists several metrics which are sensitive to imbalance problems. 
Here we will introduce the **[balanced accuracy](https://scikit-learn.org/stable/modules/model_evaluation.html#balanced-accuracy-score)**:

$$balanced\_accuracy = 0.5*( \frac{TP}{TP+FN} + \frac{TN}{TN+FP} )$$

> Other scores you may want to look-up : [average-precision score](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.average_precision_score.html#sklearn.metrics.average_precision_score), and [f1-score](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.f1_score.html#sklearn.metrics.f1_score), which are both linked to the precision/recall curve


In [ ]:
from sklearn.metrics import balanced_accuracy_score


def check_imbalance_effect( imbalance_list , class_weight = None):
    
    recall_list = []
    balanced_acc_list = []
    acc_list = []
    for imbalance in imbalance_list:

        n0 = 300
        n1 = int( n0 * (1 - imbalance) )
        if n1 == 0:
            n1 = 1

        X1 = np.concatenate( [ np.random.randn(n0) , np.random.randn(n1)+2 ])
        y = np.array( [0]*n0 + [1]*n1 )

        X1_norm = StandardScaler().fit_transform(X1.reshape( X1.shape[0] , 1 ))

        # LR
        lr = LogisticRegression( penalty = 'l2' , C = 1 , class_weight=class_weight)
        lr.fit(X1_norm , y)

        y_predicted = lr.predict(X1_norm)

        recall_list.append( recall_score( y , y_predicted )  )
        acc_list.append( accuracy_score(y,y_predicted) )
        balanced_acc_list.append( balanced_accuracy_score(y,y_predicted) )

    return recall_list , acc_list , balanced_acc_list

In [ ]:
imbalance_list = np.linspace(0,1,50)

### first, we see what happens without class_weight=None

recall_list , acc_list , balanced_acc_list = check_imbalance_effect( imbalance_list , 
                                                               class_weight = None)
    
        
fig,ax=plt.subplots(1,2,figsize = (12,4))
ax[0].plot( imbalance_list , acc_list , label='accuracy - class_weight=None' )
ax[0].plot( imbalance_list , recall_list , label='recall - class_weight=None' )
ax[0].plot( imbalance_list , balanced_acc_list , label='balanced_accuracy - class_weight=None' )
ax[0].set_xlabel("imbalance")
ax[0].set_ylim(0,1)
ax[0].legend()
## now, with class weight 

recall_list , acc_list , balanced_acc_list = check_imbalance_effect( imbalance_list , 
                                                               class_weight = 'balanced')
            
ax[1].plot( imbalance_list , acc_list , label='accuracy - class_weight=balanced' )
ax[1].plot( imbalance_list , recall_list , label='recall - class_weight=balanced' )
ax[1].plot( imbalance_list , balanced_acc_list , label='balanced_accuracy - class_weight=balanced' )
ax[1].set_xlabel("imbalance")
ax[1].set_ylim(0,1)
ax[1].legend()

So, the **balanced accuracy** is able to detect an imbalance problem.

Setting `class_weight='balanced'` in our logistic regression fixes the imbalance at the level of the model.


**micro-exercise**:  re-explore the hyper-parameters of the logisitic regression for the cancer data-set, but this time, account for the imbalance between malignant and benign samples.


<br>

---

<br>

**If you want to use a GLM other than the logistic regression:**
[GLM in sklearn](https://scikit-learn.org/stable/modules/linear_model.html#generalized-linear-regression)


<br>

[Back to the ToC](#toc)

<br>

# A few VERY IMPORTANT words on leakage. <a class="anchor" id="leakage"></a>

The most important part in all of the machine learning jobs that we have been presenting above, is that **the data set on which you train and the data set on which you evaluate your model should be clearly separated**(either the validation set when you do hyperparameter tunning, or test set for the final evaluation). 

No information directly coming from your test or your validation should pollute your train set. If it does you **loose your ablity to have a meaningful evaluation power.** 

In general **data leakage** relates to every bits of information that you should not have access to in a real case scenario, being present in your training set.

Among those examples of data leakage you could count : 
* **using performance on the test set to decide which algorithm/hyperparameter to use**
* doing imputation or scaling before the train/test split
* inclusion of future data points in a time dependent or event dependent model.


<br>

[Back to the ToC](#toc)

<br>

# Support Vector Machine <a class="anchor" id="svm"></a>

## SVM for Classification <a class="anchor" id="svm-c"></a>

### introduction <a class="anchor" id="formal-svm-c"></a>

"The basic principle of SVM is pretty simple. SVM aims at finding the 'good' threshold (hyperplane) to separate data from different classes. Conceptually it is very different from logistic regression where you maximize the log likelihood of the log odds function. **With SVM you really look for an hyperplane that separates data and that's it : there is no underlying hypothesis about probability distribution or anything else. It is very geometrical.**

So what's a good threshold? Again it is going to depend on the metric you are interested in. But at least a good threshold should be linked to this biais variance trade off in the sense that it should offer flexibility to your model.

You can imagine that there is a quite a lot of hyperplanes separating data in your training set. You could stick your threshold right where the class 0 point closest to class 1 lies. But in that case it will be very far from the other class 0 points, which can be a problem. **You could decide that your threshold should be right between the two closest extreme of your classes but that is going to be very sensitive to missclassified data or extreme events... Those points choosen as a reference to put your threshold are called support vectors.**

In [ ]:
np.random.seed(10815657)


fig,ax = plt.subplots(1,2,figsize=(15,7))

# case 1 
norm1=0.2*np.random.randn(100)-2
norm2=0.8*np.random.randn(100)+2.5
ax[0].plot(norm1,[1]*100,'ro',markersize=10)
ax[0].plot(norm2,[1]*100,'bo')

min2 = min( norm2 )
max1 = max( norm1 )

ax[0].axvline(min2, color='k', linestyle='--', label='defined by the most extreme blue point')
ax[0].axvline( (min2+max1)/2,color='k',linestyle='-.',label='middle of extreme of two classes')
ax[0].legend(loc='best')


# case 2
cauch=0.8*np.random.standard_cauchy(10)-2
norm=1*np.random.randn(100)+2.5

ax[1].plot(cauch,[1]*10,'ro',markersize=10)
ax[1].plot(norm,[1]*100,'bo')

min2 = min( norm )
max1 = max( cauch )

ax[1].axvline(min2, color='k', linestyle='--', label='defined by the most extreme blue point')
ax[1].axvline( (min2+max1)/2,color='k',linestyle='-.',label='middle of extreme of two classes')
ax[1].legend(loc='best')


On the left panel, the two hyperplanes are valid separation but you can imagine that the plane defined by the most extreme blue point doesn't leave much space for generalization

If your data are not linearly separable, like in the right panel, you need to be able to choose support vectors that are going to do some misclassification but for the greater good.

So, once again, you are confronted to a compromise. You should place your threshold somwhere that is globally best even though that would mean some miss-classification. We are back to our regularization problem and of course **Support vector machine has a regularization parameter : C**. The game now becomes placing your threshold right in the middle of points (support vectors) from  each classes that you have \"chosen\" to be general points for decision making : **they don't need to be the two closest points from different classes anymore. They need to be points where your hyperplane makes the least error differentiating classes.**


![svm](image/1920px-SVM_margin.png)

Image source : image by wikipedia user Larhmam, distributed under a [CC BY-SA 4.0 license](https://creativecommons.org/licenses/by-sa/4.0/deed.en).


So you want to maximize the margin separating the two classes. This margin is $\frac{2}{||\pmb{w}||}$. So you want to minimize $||\pmb{w}||$. The SVM loss function we want to minimize with respect to $\pmb{w}$ and $b$ is:

$C\cdot\Sigma^{N}_{i=1}\zeta_i + \frac{1}{2}||\pmb{w}||^{2}$ subject to $\zeta_i \ge 0$ and $y_{i}(w^{T}x_{i}-b) \ge 1-\zeta_i$, where $\zeta_i = \Sigma^{N}_{i=1}max(0,1-y_{i}(\pmb{w}\cdot\pmb{x}_i-b))$
 * $y_i$ is $-1$ or $1$ depending on the class of the point $i$
 * the class of point $\pmb{x}$ is determined by the SVM using the sign of $(\pmb{w}\cdot\pmb{x}-b)$ (ie, on which side of the $(\pmb{w}\cdot\pmb{x}-b)$ hyperplane we are).



Note that you could also use a L1 regularization but it is not implemented in the function we are going to use.

Indeed if most of the data points are well separated in term of class on each side of the hyperplane then
* most of the time $y_{k}(w^{T}x_{k}-b) \geq 1$ and so $max(0,1-y_{k}(w^{T}x_{k}-b)=0$ (that's good for minimizing our loss function), 
* and a few times $y_{k}(w^{T}x_{k}-b) \leq -1$ and so $max(0,1-y_{k}(w^{T}x_{k}-b) \geq 2$ (which is polluting our minimization of the loss function).



You can see that there is a [dot product](https://en.wikipedia.org/wiki/Dot_product) involved : in the case of a linear hyperplane this dot product is just the cartesian dot product that you probably use all the time. It allows you to calculate distances between points in that cartesian space or between points and hyperplanes. But you might be familiar with other scalar product : like for example when you proceed to a Fourier decomposition of a function. This particular scalar product acts on functions and so is not really of interest for us... But others exist.

**So in principle you could use other definitions of distance between points to answer that classification question**. This is what non-linear SVM does and this is why you can choose different so called kernels as hyperparameters as we will see below :

$\overrightarrow{x_{i}}.\overrightarrow{x_{j}}$ : cartesian

$(\overrightarrow{x_{i}}.\overrightarrow{x_{j}})^{d}$ : polynomial degree d

$exp(-\gamma||\overrightarrow{x_{i}}-\overrightarrow{x_{j}}||^{2})$ : gaussian radial basis

$tanh(\kappa\overrightarrow{x_{i}}.\overrightarrow{x_{j}}+c)$ : hyperbolic tangent

**This is really powerful for classification but going non-linear by using a kernel trick prevents you from interpreting how your features are massaged to create this classifier... So, if you want interpretability and do science rather than engineering : keep it linear.**

![3d_svm](image/3d_svm.png)

> Even though SVM as nothing to do with probablities, we are going to transform the results of our classifier back to probabilities (using logistic regression...) to be able to draw a ROC curve. But again I insist, those are just useful transformations but has actually nothing to do with the technique.

[Back to the toc](#toc)

### Toy example to visualize SVMC <a class="anchor" id="toy-example-svm-c"></a>

In [ ]:
from sklearn.datasets import make_blobs
X2, y2 = make_blobs(n_samples=(250,250), centers=[[-1,-1],[1,1]], random_state=6)
plt.scatter(X2[:,0],X2[:,1],c=y2)
plt.xlim(min(X2[:,1])-0.5,max(X2[:,1])+0.5)
plt.ylim(min(X2[:,1])-0.5,max(X2[:,1])+0.5)
plt.show()

In [ ]:
from utils import contour_SVM

contour_SVM(X2,y2,c=100,ker='linear')

In [ ]:
contour_SVM(X2,y2,c=0.01,ker='linear')

In [ ]:
contour_SVM(X2,y2,c=1,ker='rbf',gam=10)

In [ ]:
contour_SVM(X2,y2,c=1,ker='rbf',gam=1)

In [ ]:
contour_SVM(X2,y2,c=1,ker='poly',deg=5)

[Back to the toc](#toc)

### SVM Classifier pipeline. <a class="anchor" id="svm-c-pipeline"></a>

In sklearn SVM classifier is implemented in a single `sklearn.svm.SVC` class,
but as we have seen depending on the kernel you have different parameters.

That means that when you specify the grid of hyper-parameters to explore, you could write something like this:

```python
grid_values = {"model__kernel": ['linear', 'rbf', 'poly'],
                 "model__C":np.logspace(-2, 2, 10),
                 "model__degree":[2,3,4,5],
                 "model__gamma": np.logspace(-2,1,10)}
```

That would work, however that means that even though parameter `gamma` is useless when the kernel is `'linear'`, the GridSearchCV will still test all the different combination of values.
On this example, rather than testing :
 * 10 combinations for linear kernel (C)
 * 10*4 = 40 combinations for poly kernel (C and degree)
 * 10*10 = 100 combinations for rbf kernel (C and gamma)

= 150 combinations to test.

You would test $3*10*4*10 = 1200$ combinations...

Let's see how we can handle this smartly to avoid these useless computations.

In [ ]:
%%time
from sklearn.svm import SVC

## set up the pipeline as usual
pipe = Pipeline([('scalar',StandardScaler()),
                 ("classifier", SVC())])

# the grid of parameter values is not a dictionnary, but now a list 
#  of dictionnaries : smaller grid to explore independently

# for each of these grid, we can re-define locally the classifier
grid_param = [  {"classifier": [SVC(class_weight='balanced', probability=True, kernel='linear')],
                 "classifier__C":np.logspace(-2, 2, 10)},
                {"classifier": [SVC(class_weight='balanced', probability=True, kernel='rbf')],
                 "classifier__C":np.logspace(-2, 2, 10),
                 "classifier__gamma": np.logspace(-2,1,10)},
                {"classifier": [SVC(class_weight='balanced', probability=True, kernel='poly')],
                 "classifier__C":np.logspace(-2, 2, 10),
                 "classifier__degree":[2,3,4,5]} ]
## NB : we use probability = True in order to make ROC auc computation possible later on

grid_svm = GridSearchCV(pipe, 
                        grid_param, 
                        cv=5, 
                        verbose=0,
                        n_jobs=-1,
                        scoring='accuracy') 

grid_svm.fit(X_train_cancer, y_train_cancer)

#get the best cross-validated score 
print('Grid best score ('+grid_svm.scoring+'): ',
      grid_svm.best_score_)
# print the best parameters
print('Grid best kernel    :' , grid_svm.best_params_["classifier"].kernel )
print('Grid best parameter :')
for k,v in grid_svm.best_params_.items():
    print(' {:>20} : {}'.format(k,v))


Alternatively, the same as above could be done with multiple separate grid searchs. We would **compare them by their cross-validated score** : `grid.best_score_`.

In [ ]:
%%time
from sklearn.svm import SVC

scoring_metric = "accuracy"

# linear kernel
pipe1 = Pipeline([('scalar',StandardScaler()),
                 ("classifier", SVC(class_weight='balanced', probability=True, kernel='linear'))])
grid_param1 = { "classifier__C":np.logspace(-2, 2, 10)}

grid_svm_linear = GridSearchCV(pipe1, grid_param1, 
                        cv=5, verbose=0, n_jobs=-1, scoring=scoring_metric) 

# rbf kernel
pipe2 = Pipeline([('scalar',StandardScaler()),
                 ("classifier", SVC(class_weight='balanced', probability=True, kernel='rbf'))])
grid_param2 = { "classifier__C":np.logspace(-2, 2, 10) , 
               "classifier__gamma": np.logspace(-2,1,10)}

grid_svm_rbf = GridSearchCV(pipe2, grid_param2, 
                        cv=5, verbose=0, n_jobs=-1, scoring=scoring_metric) 

# poly kernel
pipe3 = Pipeline([('scalar',StandardScaler()),
                 ("classifier", SVC(class_weight='balanced', probability=True, kernel='poly'))])
grid_param3 = { "classifier__C":np.logspace(-2, 2, 10) , 
               "classifier__degree": [2,3,4,5]}

grid_svm_poly = GridSearchCV(pipe3, grid_param3, 
                        cv=5, verbose=0, n_jobs=-1, scoring=scoring_metric) 



grid_svm_linear.fit(X_train_cancer, y_train_cancer)
grid_svm_rbf.fit(X_train_cancer, y_train_cancer)
grid_svm_poly.fit(X_train_cancer, y_train_cancer)


print('linear Grid best score ('+grid_svm_linear.scoring+'): ',grid_svm_linear.best_score_)
print('   rbf Grid best score ('+grid_svm_rbf.scoring+'): '   ,grid_svm_rbf.best_score_)
print('  poly Grid best score ('+grid_svm_poly.scoring+'): '  ,grid_svm_poly.best_score_)

In the end we get the same result.

> This is also the approach we would take to decide between logistic regression, SVM, or other models.

Anyhow, let's look at the best model performance in more depth:

In [ ]:
y_cancer_test_score=grid_svm.score(X_test_cancer,y_test_cancer)

print('Grid best parameter (max.'+grid_svm.scoring+') model on test: ', y_cancer_test_score)

In [ ]:
y_cancer_pred_test=grid_svm.predict(X_test_cancer)

confusion_m_cancer_SVMC = confusion_matrix(y_test_cancer, y_cancer_pred_test)

plt.figure(figsize=(5,4))
sns.heatmap(confusion_m_cancer_SVMC, annot=True)

plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.title("test {} : {:.3f}".format(grid_svm.scoring , y_cancer_test_score))

In [ ]:
from scipy.special import expit

In [ ]:

y_cancer_score_SVMC = grid_svm.decision_function(X_test_cancer)
fpr_SVMC_cancer, tpr_SVMC_cancer, thre_SVMC_cancer = roc_curve(y_test_cancer, y_cancer_score_SVMC)
roc_auc_SVMC_cancer = auc(fpr_SVMC_cancer, tpr_SVMC_cancer)

proba=expit(thre_SVMC_cancer)
for i in range(len(proba)):
    if abs(proba[i]-0.5)<0.1:
        keep=i
        break
        

plt.figure()
plt.xlim([-0.01, 1.00])
plt.ylim([-0.01, 1.01])
plt.plot(fpr_SVMC_cancer, tpr_SVMC_cancer, lw=3, label='SVC ROC curve\n (area = {:0.2f})'.format(roc_auc_SVMC_cancer))
plt.plot(fpr_SVMC_cancer[keep], tpr_SVMC_cancer[keep],'ro',label='threshold=0.5')
plt.xlabel('False Positive Rate', fontsize=16)
plt.ylabel('True Positive Rate', fontsize=16)
plt.title('ROC curve (SVM classifier)', fontsize=16)
plt.legend(loc='lower right', fontsize=13)
plt.plot([0, 1], [0, 1], color='navy', lw=3, linestyle='--')
#plt.axes().set_aspect('equal')
plt.show()

And here the kernel is RBF, so we have no coefficients to grab from the model for interpretation.

But, we still have some options.

For instance, we present here [permutation feature importance](https://scikit-learn.org/stable/modules/permutation_importance.html): it is the **decrease in a model score when a single feature value is randomly shuffled**.


In [ ]:
%%time
from sklearn.inspection import permutation_importance

r = permutation_importance(grid_svm.best_estimator_, X_test_cancer, y_test_cancer,
                            n_repeats=1000,
                            random_state=132987)

for i in r.importances_mean.argsort()[::-1]:
    if r.importances_mean[i] - 2 * r.importances_std[i] > 0:
        print(f"{breast_cancer_df.columns[i]:>25} : "
              f"{r.importances_mean[i]:.3f}"
              f" +/- {r.importances_std[i]:.3f}")


Other methods exists to help analyse black-boxxy models, such as [SHAP](https://shap.readthedocs.io/en/latest/example_notebooks/tabular_examples/model_agnostic/Iris%20classification%20with%20scikit-learn.html)

<br>

[Back to the ToC](#toc)

<br>

## SVM for Regression <a class="anchor" id="svm-r"></a>

We could use the same algorithm for regression, with the only difference that this time instead of finding the hyperplane that is the farthest from the support, we find the hyperplane that is the closest from those support. 

For example, on our diabete data set, just by replacing SVC by SVR. 
We just use the `'rbf'` kernel (the linear and poly have already been tested with our OLS model earlier).

In [ ]:
X_potato_train = dfTT.loc[i1 , :]
y_potato_train = df.loc[i1 , "Flesh Colour"]

X_potato_test = dfTT.loc[i2 , :]
y_potato_test = df.loc[i2 , "Flesh Colour"]


In [ ]:
%%time
from sklearn.svm import SVR
## SVR for regression

pipeline_SVR=Pipeline([('scalar',StandardScaler()),
                       ('model',SVR(kernel='rbf'))])

grid_values = {"model__C":np.logspace(-0, 4, 25),
               "model__gamma": np.logspace(-6,-4,25)}

## don't forget to change the metric to one adapted for regression:
grid_SVR = GridSearchCV(pipeline_SVR, 
                                 param_grid = grid_values, 
                                 scoring='r2', n_jobs=-1)

grid_SVR.fit(X_potato_train, y_potato_train)

print('Grid best score ('+grid_SVR.scoring+'): ', grid_SVR.best_score_)
print('Grid best parameter (max.'+grid_SVR.scoring+'): ', grid_SVR.best_params_)


If you remember, our OLS model was able to get an $R^2$ of $~0.592$.

So we do not gain any $R^2$, and also we loose interpretability ... not the best trade here.

Let's still have a look at the model predictions : 

In [ ]:
y_potato_test_score=grid_SVR.score(X_potato_test,y_potato_test)
print('Grid best parameter (max.'+grid_SVR.scoring+') model on test: ', y_potato_test_score)

In [ ]:
## train prediction
y_train_pred = grid_SVR.predict(X_potato_train)
## test prediction
y_test_pred  = grid_SVR.predict(X_potato_test)


plt.scatter( y_train_pred , y_potato_train , c="blue" , label='train set'  )
plt.scatter( y_test_pred , y_potato_test , c="red" , label='test set'  )
m,M = min(y_train_pred) , max(y_train_pred)
plt.plot( [m,M] , [m,M] , 'k--' )
plt.xlabel( 'predicted values' ) 
plt.ylabel( 'real values' ) 
plt.legend()

<br>

<!-- Exo : try a  linear kernel. Can you say anything about feature importance? How would you compare this new model to the former. %load  solutions/solution_03_mini.py -->

 ---

<br>

[Back to the ToC](#toc)

<br>

# Decision tree modeling : a (new?) loss function and new ways to do regularization. <a class="anchor" id="decision-tree"></a>

## Simple decision tree for classification <a class="anchor" id="simple-tree-c"></a>

A simple **decision tree** reduces your problem into a **hierarchichal sequence of questions** on your features that can be answered by yes or no and which subdivides the data into 2 subgroups on which a new question is asked, and so on and so on.
![tree_ex](image/tree_ex.png)

Ok, but a huge number of trees can actually be built just by considering the different orders of questions asked. How does the algorithm deals with this?

Quite simply actually: it **tests all the features and chooses the most discriminative** (with respect to your target variable) : the feature where a yes or no question divides the data into 2 subsets which minimizes an **impurity measure**.

Imagine you have a dataset with feature color (red or blue) and feature shape (square or circle), and 2 target classes : 1 and 2.


![tree](image/Tree.png)

Asking `"feature color is red"` gives you the following subgroups:
 * 10 class 1, and 1 class 2 (`"feature color is red" == True`)
 * 2 class 1, and 11 class 2 (`"feature color is red" == False`)

Asking `"feature shape is square"` gives you:
 * 5 class 1, and 7 class 2 (`True`) 
 * 7 class 1 and 5 class 2 (`False`)
 
 So, you will prefer asking `"feature color is red?"` over `"feature shape is square?"`: `"feature color is red?"` is more discriminative.

For **categorical variables, the questions test for a specific category**.
For **numerical variables, the questions use a threshold** to as a yes/no question.  

The **threshold is, again, chosen to minimize impurity**. And in turn the best threshold for a variable is used to estimate the discriminativeness of that variable.

Of course, you will have to compute this threshold at each step of your tree since at each step you are considering different subdatasets.

---
The **impurity is related to how much your feature splitting is still having mixed classes**. So the impurity ends up giving a score: either it is a simple [Shannon entropy](https://en.wikipedia.org/wiki/Entropy_(information_theory)) or it is a [Gini coefficient](https://en.wikipedia.org/wiki/Gini_coefficient).

#### Shannon Entropy

$$Entropy = - \sum_{j} p_j log_2(p_j)$$

This measure is linked to information theory, where the information of an event occuring is the $log_2$ of this event's probability of occuring.
For purity, **0 is the best possible score, and 1 the worst**.

#### Gini coefficient

$$Gini = 1- \sum_{j} p_j^2$$

The idea is to measure the **probability that a dummy classifier mislabels your data**.
**0 is best, 1 is worst.**

---
Before going further, just a little bit of vocabulary: 
* **Trees** are made of **nodes** (where the question is asked and where the splitting occurs). 
* A **branch** is the outcome of a splitting. 
* A **leaf** is the last node on a branch (no more splitting).

[Back to the ToC](#toc)

### Toy example to visualize decision tree. <a class="anchor" id="toy-decision-tree"></a>

Let explore some hyperparameters of this method that, you will see in those examples, act like a regularization:
- **Max Tree depth**: the maximum number of consecutive questions to ask
- **Min Splitting of nodes**: minimum number of points to consider to make a new rule, outside of the leaves
- **Min Splitting of leaves**: minimum number of points to consider to make a new rule, at the leaves

In [ ]:
X_3, y_3 = make_blobs(n_samples=120, centers=3,cluster_std=[[1,3],[1,3],[1,3]], random_state=6)
plt.scatter(X_3[:,0],X_3[:,1],c=y_3,cmap=plt.cm.coolwarm,edgecolors='k')

In [ ]:
from sklearn.tree import DecisionTreeClassifier
## creating a decision tree with 1 parameter changed (more on that later)
tree = DecisionTreeClassifier(max_depth=3)
tree.fit(X_3, y_3)

In [ ]:
pd.crosstab( tree.predict( X_3 ) , y_3 , rownames=['truth'] , colnames=['prediction'] )

In [ ]:
from sklearn.tree import plot_tree
fig,ax = plt.subplots(figsize=(14,6))

_ = plot_tree( tree , feature_names=['x','y'] , 
               fontsize=14 , filled=True , impurity=False , precision=3, ax=ax)

In [ ]:
from utils import contour_tree
contour_tree(X_3, y_3,
              crit = 'entropy',
              maxd = None,
              min_s = 2,
              min_l = 1,
              max_f = None)
#You can see that there are 5 hyperparameters here. Let's see what they do and what they mean.
#I bet you can already guess it is going to be related to regularization....
# After X,y you have 
# * crit = 'entropy' which is one way to calculate impurity (you could also put gini here)
# * maxd : the max depth of your tree
# * min_s : the number of points that should be concerned by the making of a new rule (splitting of the nodes)
# * min_l : #of points that should be considered to make a final leaf classification
# * max_f maximum number of features to consider for making a new rule...

This is an incredibly complex model. 

Please, note that since every node is a question asked on one particular feature and features are never directly compared, you don't need scaling! This observation that each question always involves one feature at a time can be also seen in the way the boundaries between classes are made in the graph : there is no diagonal boundaries. You can only see lines parallel to the plot axes.

In [ ]:
#using another impurity measurement
contour_tree(X_3, y_3,
              crit = 'gini',
              maxd = None,
              min_s = 2,
              min_l = 1,
              max_f = None)

Still some overfitting but it is nice to see that the boundaries are different and that impurity calculations, even if very similar, are making a difference.

In [ ]:
#Imposing a limit for the depth of the tree : how many questions you ask (here set to 4)
contour_tree(X_3, y_3,
              crit = 'entropy',
              maxd = 4,
              min_s = 2, min_l = 1, max_f = None)

In [ ]:
contour_tree(X_3, y_3,
              crit = 'entropy',
              maxd = None,
              min_s = 2,
              min_l = 4,
              max_f = None)
# min_samples_leaf : 
#     it sets the minimal number of data points that the chain of rules should concern. 

# eg. Do you really wish to create a whole new set of rules to explain 
# only one particular data point? 

In [ ]:
contour_tree(X_3, y_3,
              crit = 'entropy',
              maxd = None,
              min_s = 20,
              min_l = 1,
              max_f = None)
# Here it is the same as before but this time it applies to nodes instead of leaves
# This parameter is called min_samples_split and is set to 20 here.

There are 3 main advantages to this kind of methods:
* it works with all types of feature
* you don't need to rescale
* it already includes non linear fitting

**Moreover it is 'easy' to interpret.**

But....(yes there is a but, there is no free lunch)

Even with all of those hyperparamaters **they are still not great on new data (inaccuracy...).** 

We will see that in the real data example below and we will see more powerful technics based on decision tree that are more costly but generalize better.

[Back to the ToC](#toc)

### Single decision tree pipeline. <a class="anchor" id="single-tree-pipeline"></a>

In [ ]:
## the different hyper parameters on the decision tree are quite related to the dataset size, 
##  both in number of columns (for max depth)
##  and in number of rows (for min sample split and min sample leaf)

X_train_cancer.shape

In [ ]:
%%time
from sklearn.tree import DecisionTreeClassifier

grid_values = {'criterion': ['entropy','gini'],
               'max_depth':np.arange(2,10),
               'min_samples_split':np.arange(2,12,2)
              }

grid_tree_cancer = GridSearchCV(DecisionTreeClassifier(class_weight="balanced"), 
                                param_grid = grid_values, 
                                scoring='accuracy',
                                n_jobs=-1)
grid_tree_cancer.fit(X_train_cancer, y_train_cancer)

print(f'Grid best score (accuracy): {grid_tree_cancer.best_score_:.3f}')
print('Grid best parameter :')

for k,v in grid_tree_cancer.best_params_.items():
    print('{:>25}\t{}'.format(k,v))

In [ ]:
y_cancer_test_score=grid_tree_cancer.score(X_test_cancer,y_test_cancer)

print('Grid best parameter (max. accuracy) model on test: ', y_cancer_test_score)

y_cancer_pred_test = grid_tree_cancer.predict(X_test_cancer)

confusion_m_cancer_tree = confusion_matrix(y_test_cancer, y_cancer_pred_test)
plt.figure(figsize=(5.5,4))
sns.heatmap(confusion_m_cancer_tree, annot=True)
plt.title('test {} : {:.3f}'.format( grid_tree_cancer.scoring , y_cancer_test_score ))
plt.ylabel('True label')
plt.xlabel('Predicted label')

Feature importance can be retrieved from the tree: 

In [ ]:
w_tree=grid_tree_cancer.best_estimator_.feature_importances_

sorted_features=sorted([[breast_cancer_df.columns[i],abs(w_tree[i])] for i in range(len(w_tree))],
                       key=itemgetter(1),reverse=True)

print('Features sorted per importance in discriminative process')
for f,w in sorted_features:
    print('{:>25}\t{:.3f}'.format(f,w))

And we can even plot the model:

In [ ]:
from sklearn.tree import plot_tree
fig,ax = plt.subplots(figsize=(25,10))
plot_tree( grid_tree_cancer.best_estimator_ , 
          feature_names=breast_cancer_df.columns , 
          ax=ax , fontsize=14 , filled=True , impurity=False , precision=3)
ax.set_title('best single decision tree')

<br>

[Back to the ToC](#toc)

<br>

## Random Forest in classification. <a class="anchor" id="rf-c"></a>

the Random Forest algorithm relies on two main concepts : 
1. **randomly producing/training many different trees**
2. **agglomerating the predictions** of all these trees (mainly averaging)


The randomness between trees concerns:
* **[bootstrapping](https://en.wikipedia.org/wiki/Bootstrapping_(statistics)) of the training dataset**
* using only a **random subset of features**


**Bootstrapping:** sampling methods in which you randomly draw a subsample from your data, *with replacement*. The created replicate is the same size as the original distribution.

I am sure you can see intuitively how that is going to help generalization of our model.

So now on top of all the parameters seen before to create each individual trees of the forest, you also have a parameter controlling the number of trees in your forest.

![RF](image/RF.png)
**In the following plots I am plotting the result for a random forest algorithm and compare it to a single decision tree sharing the same hyperparameters value than the one used in the random forest**.




In [ ]:
from utils import contour_RF

contour_RF(X_3,y_3,n_tree = 200, 
            crit = 'gini', maxd = 4,min_s = 5, min_l = 5, max_f = 'sqrt')
#Same as for decision tree except that we have here one more hyperparameter, here
# put to 100 and that represents the number of bootstraps 
# (number of trees trained and then participating to the vote)

# also, we restrict the number of variables given to each tree to 
# the square root of the original number of variables ->  max_f = 'sqrt'

[Back to the ToC](#toc)


### Exercise: Random Forest on the breast cancer dataset <a class="anchor" id="rf-exo"></a>

Train a random forest on the breast cancer dataset.

Use an hyper-parameter space similar to the one we used for single decision trees with the number of trees (`n_estimator`) added to it.

**computational considerations**: to limit the training time to around 1 minute, only test 5 values of `n_estimators`, all below 500.

correction:

In [ ]:
# %load solutions/solution_03_RF_cancer.py
from sklearn.ensemble import RandomForestClassifier

grid_values = {'n_estimators' : [10,50,100,150,200], 
               'criterion': ['entropy','gini'],
               'max_depth':np.arange(2,10), ## I reduce the search space in the interest of time too
               'min_samples_split':np.arange(2,12,2)}

grid_tree = GridSearchCV(RandomForestClassifier(class_weight='balanced'), 
                                param_grid = grid_values, 
                                scoring='roc_auc',
                                cv = 5,
                                n_jobs=-1)
grid_tree.fit(X_train_cancer, y_train_cancer)

print(f'Grid best score (accuracy): {grid_tree.best_score_:.3f}')
print('Grid best parameter :')

for k,v in grid_tree.best_params_.items():
    print('{:>25}\t{}'.format(k,v))

We can also get a Receiver-Operator curve from this model:

In [ ]:
from sklearn.metrics import roc_curve
y_test_score=grid_tree.predict_proba(X_test_cancer)[:,1]

fpr, tpr, thresholds = roc_curve(y_test_cancer, y_test_score)

keep = sum( thresholds > 0.5 ) - 1 # trick to find the index of the last threshold > 0.5

y_test_roc_auc = grid_tree.score(X_test_cancer,y_test_cancer)

plt.figure()
plt.xlim([-0.01, 1.01])
plt.ylim([-0.01, 1.01])
plt.plot(fpr, tpr, lw=3)
plt.plot(fpr[keep], tpr[keep],'ro',label='threshold=0.5')
plt.xlabel('False Positive Rate', fontsize=16)
plt.ylabel('True Positive Rate', fontsize=16)
plt.title(f'ROC AUC (Decision tree) {y_test_roc_auc:.3f}', fontsize=16)
plt.legend(loc='lower right', fontsize=13)
plt.plot([0, 1], [0, 1], color='navy', lw=3, linestyle='--')
plt.show()

Trees do not have coefficients like the logistic regression, but they still have a feature importance metric which is computed from how much each feature reduce impurity. 

In [ ]:
w_tree=grid_tree.best_estimator_.feature_importances_

sorted_features=sorted([[breast_cancer_df.columns[i],abs(w_tree[i])] for i in range(len(w_tree))],
                       key=lambda x : x[1],
                       reverse=True)

print('Features sorted per importance in discriminative process')
for f,w in sorted_features:
    if w == 0:
        break
    print('{:>25}\t{:.4f}'.format(f,w))

In [ ]:
feature_importance = grid_tree.best_estimator_.feature_importances_

## by gathering the importance accross each individual tree, we can access 
## the standard deviation of this importance
feature_importance_std = np.std([tree.feature_importances_ for tree in grid_tree.best_estimator_.estimators_], 
                       axis=0)

sorted_idx = np.argsort(feature_importance)
pos = np.arange(sorted_idx.shape[0]) + .5
fig = plt.figure(figsize=(12, 12))


plt.barh(pos, feature_importance[sorted_idx],xerr=feature_importance_std[sorted_idx][::-1], align='center')
plt.yticks(pos, breast_cancer_df.columns[sorted_idx])
plt.title('Feature Importance (MDI)',fontsize=10)
plt.xlabel("Mean decrease in impurity")
plt.show()

[Back to the ToC](#toc)

### RF annex 1: too many features <a class="anchor" id="rf-a1"></a>

Modern biological dataset using high-throughput technologies can now provide us with measurements for hundreds or even thousands of features (e.g., proteomics, RNAseq experiments).
But it is often the case that among these thousands of features, only a handful are truly informative (the so-called biomarkers for example).

While they generally are very good methods, Random Forest can sometime struggle in this context. 
Let's try to understand why with a synthetic example:


We start with a simple case : 2 categories, perfectly separable using 2 features:

In [ ]:
from sklearn.datasets import make_blobs
    
# 120 points, 2 blobs/clusters with some spread=3
blob_centers = np.array([[0,0],[8,4]])
blob_stds = [[2,2],[2,2]]
X_2, y_2 = make_blobs(n_samples = 120, 
                      centers = blob_centers,
                      cluster_std = blob_stds, random_state = 42)

plt.scatter(X_2[:,0],X_2[:,1],c=y_2,cmap=plt.cm.coolwarm,edgecolors='k')

Let's see how a single decision tree and a random forest do in this situation:

In [ ]:
dt = DecisionTreeClassifier()
rf = RandomForestClassifier(n_estimators=100)

In [ ]:
from sklearn.model_selection import cross_val_score

print("decision tree cross-validated accuracy:" , cross_val_score( dt , X_2 , y_2 ) )
print("random forest cross-validated accuracy:" , cross_val_score( rf , X_2 , y_2 ) )

We can see that they both perform very well, perhaps even better in the case of the random forest (it is able to find more generalizable separation rules thanks to the ensembling).

Now, we are going to add many new features filled with random data (imagine they are the 99% of genes which are not biomarkers):

In [ ]:
nb_noise = 10**4
X_2_noise = np.concatenate( [ X_2 , np.random.randn( X_2.shape[0],nb_noise ) ] , axis = 1)

In [ ]:
print("decision tree cross-validated accuracy:" , cross_val_score( dt , X_2_noise , y_2 ) )
print("random forest cross-validated accuracy:" , cross_val_score( rf , X_2_noise , y_2 ) )

The performance of the **single decision tree is unchanged**,

but the **Random Forest performs way worse**!


**Question:** how can we explain this difference?


---

**Solution and discussion:**

In [ ]:
# %load solutions/solution_03_RF_skb.py

<br>

[Back to the ToC](#toc)

<br>

## Random Forest in regression. <a class="anchor" id="rf-r"></a>

From the standpoint of tree, the only difference is that now, instead of the entropy or Gini criterion, **the decision which variable to use at any node is made using a regression metric**, such as squared error for example.

For example, consider this example of [regression with a single tree](https://scikit-learn.org/stable/auto_examples/tree/plot_tree_regression.html), adapted from the sklearn website:

In [ ]:
from sklearn.tree import DecisionTreeRegressor
# Create a random dataset
rng = np.random.RandomState(1)
X = np.sort(5 * rng.rand(80, 1), axis=0)
y = np.sin(X).ravel()
y[::5] += 3 * (0.5 - rng.rand(16)) # adding additional noise to some of the points

# Fit regression model
regr_1 = DecisionTreeRegressor(max_depth=2)
regr_2 = DecisionTreeRegressor(max_depth=5)
regr_1.fit(X, y)
regr_2.fit(X, y)

# Predict
X_test = np.arange(0.0, 5.0, 0.01)[:, np.newaxis]
y_1 = regr_1.predict(X_test)
y_2 = regr_2.predict(X_test)

# Plot the results
plt.figure(figsize = (14,6))
plt.scatter(X, y, s=20, edgecolor="black", c="darkorange", label="data")
plt.plot(X_test, y_1, color="cornflowerblue", label="max_depth=2", linewidth=2)
plt.plot(X_test, y_2, color="yellowgreen", label="max_depth=5", linewidth=2)
plt.xlabel("data")
plt.ylabel("target")
plt.title("Decision Tree Regression")
plt.legend()
plt.show()


In [ ]:
fig,ax = plt.subplots(figsize=(10,5))
plot_tree( regr_1 , 
          ax=ax , fontsize=10 , filled=True , impurity=False , precision=3)
ax.set_title('best single decision tree')

Of course with a single tree you do not get very far, unless the tree becomes absolutely huge. 

But with a random forest you can aggregate the estimate from many trees to get somewhere nice.

In [ ]:
from sklearn.ensemble import RandomForestRegressor

RFReg = RandomForestRegressor(n_estimators=100 )
RFReg.fit(X, y)

# Predict
X_test = np.arange(0.0, 5.0, 0.01)[:, np.newaxis]
y_1 = regr_1.predict(X_test)
y_rf = RFReg.predict(X_test)

# Plot the results
plt.figure(figsize = (14,6))
plt.scatter(X, y, s=20, edgecolor="black", c="darkorange", label="data")
plt.plot(X_test, y_1, color="cornflowerblue", label="max_depth=2", linewidth=2)
plt.plot(X_test, y_rf, color="yellowgreen", label="RF", linewidth=2)
plt.xlabel("data")
plt.ylabel("target")
plt.title("Decision Tree Regression")
plt.legend()
plt.show()



With a bit of leg-work, we can even grab the inidividual trees predictions to build an interval around the random forest prediction: 

In [ ]:

## collecting prediction from all individual trees in a big list
y_pred = []
x_pred = []
for tree in RFReg.estimators_ :
    y_pred += list( tree.predict(X_test) )
    x_pred += list(X_test[:,0])


plt.figure(figsize = (14,6))
plt.scatter(X, y, s=20, edgecolor="black", c="darkorange", label="data")
plt.plot(X_test, y_1, color="cornflowerblue", label="max_depth=2", linewidth=2)
plt.plot(X_test, y_rf, color="yellowgreen", label="RF", linewidth=2)
sns.lineplot(x=x_pred , y=y_pred , color="yellowgreen" , errorbar = 'sd') 
plt.xlabel("data")
plt.ylabel("target")
plt.title("Decision Tree Regression")
plt.legend()
plt.show()


Let's try on the potato data:

In [ ]:
## full dataset
X = dfTT
y = df[ "Flesh Colour"]

#We start by splitting our data in a train and a test set


X_train , X_test , y_train, y_test = train_test_split(X,y , test_size=0.2)

print('train set size:',len(y_train))
print(' test set size:',len(y_test))

In [ ]:
%%time
from sklearn.ensemble import RandomForestRegressor

## when it comes to criterion, we can now choose:
# * “squared_error” (default) for the mean squared error, minimizes the L2 loss
#                                           using the mean of each terminal node,
# * “friedman_mse”, which uses mean squared error with Friedman’s improvement score for potential splits
# * “absolute_error” for the mean absolute error, which minimizes the L1 loss
#                                           using the median of each terminal node,
# * “poisson” which uses reduction in Poisson deviance to find splits.
#
# let's try squared error and absolute error

grid_values = {'criterion': ['squared_error' , 'absolute_error'],
               'n_estimators':[500], 
               'max_depth':[10,15,20],
               'min_samples_split':np.arange(2,12,2)}

grid_RF = GridSearchCV(RandomForestRegressor(),
                        param_grid = grid_values, 
                        scoring='r2',n_jobs=-1,cv=5)

grid_RF.fit(X_train, y_train)


print(f'Grid best score (r2): {grid_RF.best_score_:3f}')
print('Grid best parameter (max. r2): ')

for k , v in grid_RF.best_params_.items():
    print(f'{k:>20} : {v}')

In [ ]:
print(f'Grid best parameter (max. r2) model on test: {grid_RF.score(X_test,y_test):.3f}')

In [ ]:
feature_importance = grid_RF.best_estimator_.feature_importances_

sorted_features=sorted([[X_train.columns[i],abs(feature_importance[i])] for i in range(len(feature_importance))],
                       key= lambda x : x[1],
                       reverse=True)

print('Features sorted per importance in discriminative process')
for f,w in sorted_features:
    print('{:>20}\t{:.3f}'.format(f,w))

Tree-based techniques are interesting because:
 * they do not necessitate scaling
 * they give interpretable models and results
 * they model arbitrary non-linear problems
 
However as you have seen they tend to take longer to train...

[back to the ToC](#toc)
    
# Conclusion <a id='conclusion'></a>

During this notebook we have only given a whirlwind tour of what ML is and what is it about.

We have of course only mentionned a handful of the numerous algorithms that can be used, both for [classification and for regression](https://scikit-learn.org/stable/supervised_learning.html) (NB: this link is not an exhaustive list, just what has been implemented in the sklearn library).

However, more than a collection of algorithm, Machine Learning should also be seen as a set of methods to solve some important statistical problems :
 * **regularization** parameters (such as l1 or l2 norm, or max depth), to handle **overfitting**
 * **cross-validation** strategies, to detect **overfitting** and handle **model-selection**
 * **adapted metrics** to handle the specific of our goal and our data (handle imbalance for example).
   * [classification metrics](https://scikit-learn.org/stable/modules/model_evaluation.html#classification-metrics)
   * [regression metrics](https://scikit-learn.org/stable/modules/model_evaluation.html#regression-metrics)

<br>

[Back to the ToC](#toc)

<br>

# Classification exercise : predicting heart disease on the framingham data-set <a class="anchor" id="exo-classif"></a>

Use everything you have learned to model and predict the column `'TenYearCHD'` (dependent variable : ten year risk of coronary heart disease).

 * choose a metric appropriate to the data/question at hand
 * choose :
     * the models which you want to test (don't forget eventual pre-processing steps)
     * the hyper-parameters and their values which you want to test
 * use cross-validation pick the best model and hyper-parameter sets
 * evaluate your best model on the test set
 
> NB: with the complement of models you have, it is unlikely you will get a very high performance here. 

In [ ]:
df_heart = pd.read_csv("data/framingham.csv")
df_heart = df_heart.dropna() # dropping the rows containing some missing data

In [ ]:
##separation in X and y
X_heart = df_heart.drop( columns = "TenYearCHD" )
y_heart = df_heart[ "TenYearCHD" ]


In [ ]:
X_heart.head()

Splitting in train/test set

In [ ]:
# %load -r -7 solutions/solution_03_03.py

Logistic regression

In [ ]:
# %load -r 9-34 solutions/solution_03_03.py

SVM

In [ ]:
# %load -r 34-47 solutions/solution_03_03.py

random forest

In [ ]:
# %load -r 48-65 solutions/solution_03_03.py

Evaluation of the best model on the test set

In [ ]:
# %load -r 67-91 solutions/solution_03_03.py

ROC curve

In [ ]:
# %load -r 93-118 solutions/solution_03_03.py

getting the most important features

In [ ]:
# %load -r 119-145 solutions/solution_03_03.py

Additionnal little diagnostic plot

In [ ]:
# %load -r 146- solutions/solution_03_03.py

<br>

[Back to the ToC](#toc)

<br>

# Additionnal Regression exercise : predicting daily maximal temperature <a class="anchor" id="exo-regression"></a>

In [ ]:
features = pd.read_csv('data/One_hot_temp.csv')
features.head(5)


 * year: 2016 for all data points
 * month: number for month of the year
 * day: number for day of the year
 * week: day of the week as a character string
 * temp_2: max temperature 2 days prior
 * temp_1: max temperature 1 day prior
 * average: historical average max temperature
 * actual: max temperature measurement
 * friend: your friend’s prediction, a random number between 20 below the average and 20 above the average
 

Additionally, all the features noted forecast are weather forecast given by some organisation for that day.


We want to predict `actual`, th actual max temperature of a day.

Use a random forest to do so. You can inspire yourself from the examples of code above.

Here are a couple of plots to get you started with the data exploration:

In [ ]:
import datetime
feature_list=list(features.columns)
labels=features["actual"]
# Dates of training values
months = np.array(features)[:, feature_list.index('month')]
days = np.array(features)[:, feature_list.index('day')]
years = np.array(features)[:, feature_list.index('year')]

# List and then convert to datetime object
dates = [str(int(year)) + '-' + str(int(month)) + '-' + str(int(day)) for year, month, day in zip(years, months, days)]
dates = [datetime.datetime.strptime(date, '%Y-%m-%d') for date in dates]

# Dataframe with true values and dates
true_data = pd.DataFrame(data = {'date': dates, 'actual': labels})


plt.xlabel('Date'); 
plt.ylabel('Maximum Temperature (F)')

# Plot the actual values
plt.plot(true_data['date'], true_data['actual'], 'b-', label = 'actual')
plt.xticks(rotation = 45.)
plt.show()

In [ ]:
import datetime
feature_list=list(features.columns)
labels=features["average"]
# Dates of training values
months = np.array(features)[:, feature_list.index('month')]
days = np.array(features)[:, feature_list.index('day')]
years = np.array(features)[:, feature_list.index('year')]

# List and then convert to datetime object
dates = [str(int(year)) + '-' + str(int(month)) + '-' + str(int(day)) for year, month, day in zip(years, months, days)]
dates = [datetime.datetime.strptime(date, '%Y-%m-%d') for date in dates]

# Dataframe with true values and dates
true_data = pd.DataFrame(data = {'date': dates, 'average': labels})


plt.xlabel('Date'); 
plt.ylabel('Maximum Temperature (F)')

# Plot the average values
plt.plot(true_data['date'], true_data['average'], 'b-', label = 'average')
plt.xticks(rotation = 45. )
plt.show()

Solution - Read in data 

In [ ]:
# %load -r 1-5 solutions/solution_03_02.py
from sklearn.ensemble import RandomForestRegressor

## Read in data as pandas dataframe 
features = pd.read_csv('data/One_hot_temp.csv')
features.head()

Solution - train/test split

In [ ]:
# %load -r 8-17 solutions/solution_03_02.py
## train/test split
y = np.array(features['actual'])
# Remove the labels from the features
# axis 1 refers to the columns
X= features.drop(['Unnamed: 0', 'year', 'month', 'day',
       'actual', 'week_Fri', 'week_Mon', 'week_Sat', 'week_Sun', 'week_Thurs',
       'week_Tues', 'week_Wed'], axis = 1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25,random_state = 42)


Solution - setup and fit pipeline

In [ ]:
# %load -r 19-34 solutions/solution_03_02.py
## setup and fit pipeline
grid_values = {'criterion': ['squared_error'],
               'n_estimators':[300,600,900], 
               'max_depth':[2,5,7],
               'min_samples_split':[4],
              'min_samples_leaf':[2]}# define the hyperparameters you want to test
#with the range over which you want it to be tested.

grid_tree_acc = GridSearchCV(RandomForestRegressor(), param_grid = grid_values, scoring='r2',n_jobs=-1)#Feed it to the GridSearchCV with the right
#score over which the decision should be taken

grid_tree_acc.fit(X_train, y_train)

print('Grid best parameter (max. r2): ', grid_tree_acc.best_params_)#get the best parameters
print('Grid best score (r2): ', grid_tree_acc.best_score_)#get the best score calculated from the train/validation
#dataset

Solution - evaluate the model on the test set

In [ ]:
# %load -r 36-40 solutions/solution_03_02.py
## evaluate the model on the test set
# get the equivalent score on the test dataset : again this is the important metric
y_decision_fn_scores_acc=grid_tree_acc.score(X_test,y_test)
print('Grid best parameter (max. r2) model on test: ', y_decision_fn_scores_acc)


Solution - get the feature importances 

In [ ]:
# %load -r 41-49 solutions/solution_03_02.py
## get the feature importances 
w=grid_tree_acc.best_estimator_.feature_importances_#get the weights

sorted_features=sorted([[list(X.columns)[i],abs(w[i])] for i in range(len(w))],key=itemgetter(1),reverse=True)

print('Features sorted per importance in discriminative process')
for f,w in sorted_features:
    print('{:>20}\t{:.3f}'.format(f,w))


Solution - using permutation to get the importances



In [ ]:
# %load -r 50-73 solutions/solution_03_02.py
## using permutation to get the importances
from sklearn.inspection import permutation_importance
feature_importance = grid_tree_acc.best_estimator_.feature_importances_
std = np.std([tree.feature_importances_ for tree in grid_tree_acc.best_estimator_.estimators_], axis=0)

sorted_idx = np.argsort(feature_importance)
pos = np.arange(sorted_idx.shape[0]) + .5
fig = plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.barh(pos, feature_importance[sorted_idx],xerr=std[sorted_idx][::-1], align='center')
plt.yticks(pos, np.array(list(X.columns))[sorted_idx])
plt.title('Feature Importance (MDI)',fontsize=10)

result = permutation_importance(grid_tree_acc.best_estimator_, 
                                X_test, y_test, n_repeats=10,
                                random_state=42, n_jobs=2)

sorted_idx = result.importances_mean.argsort()
plt.subplot(1, 2, 2)
plt.boxplot(result.importances[sorted_idx].T,
            vert=False, labels=np.array(list(X.columns))[sorted_idx])
plt.title("Permutation Importance (test set)",fontsize=10)
fig.tight_layout()
plt.show()

Solution - BONUS - re-thinking the splitting strategy

![RF](image/TimeSeriesSplit.png)

In [ ]:
# %load solutions/solution_03_02ter.py
## Our splitting strategy doesn't seem to represent the reality of the process....
## inspired from https://hub.packtpub.com/cross-validation-strategies-for-time-series-forecasting-tutorial/

import scipy as sc
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import TimeSeriesSplit
y = np.array(features['actual'])

# Remove the labels from the features
# axis 1 refers to the columns
X= features.drop(['Unnamed: 0', 'year', 'month', 'day',
       'actual', 'forecast_noaa', 'forecast_acc', 'forecast_under',
       'week_Fri', 'week_Mon', 'week_Sat', 'week_Sun', 'week_Thurs',
       'week_Tues', 'week_Wed'], axis = 1)

## the train data is the 75% most ancient data, the test is the 25% most recent
X_train=np.array(X)[:int(len(X.index)*0.75),:]                                                                           
X_test=np.array(X)[int(len(X.index)*0.75):,:]
y_train=np.array(y)[:int(len(X.index)*0.75)]
y_test=np.array(y)[int(len(X.index)*0.75):]

grid_values = {'criterion': ['squared_error'],
               'n_estimators':[300,600,900], 
               'max_depth':[2,5,7],
               'min_samples_split':[4],
              'min_samples_leaf':[2]}# define the hyperparameters you want to test

#with the range over which you want it to be tested.
tscv = TimeSeriesSplit()
    
#Feed it to the GridSearchCV with the right
#score over which the decision should be taken    
grid_tree_acc = GridSearchCV(RandomForestRegressor(), 
                            param_grid = grid_values, 
                            scoring='r2',
                            cv=tscv,
                            n_jobs=-1)


grid_tree_acc.fit(X_train, y_train)



print('Grid best parameter (max. r2): ', grid_tree_acc.best_params_)#get the best parameters
print('Grid best score (r2): ', grid_tree_acc.best_score_)#get the best score calculated from the train/validation
#dataset



y_decision_fn_scores_acc=grid_tree_acc.score(X_test,y_test)
print('Grid best parameter (max. r2) model on test: ', y_decision_fn_scores_acc)# get the equivalent score on the test
#dataset : again this is the important metric


## feature importances
RF = grid_tree_acc.best_estimator_
W=RF.feature_importances_#get the weights

sorted_features=sorted([[list(X.columns)[i],abs(W[i])] for i in range(len(W))],key=itemgetter(1),reverse=True)

print('Features sorted per importance in discriminative process')
for f,w in sorted_features:
    print('{:>20}\t{:.3f}'.format(f,w))
    
from sklearn.inspection import permutation_importance

feature_importance = RF.feature_importances_#get the weights
std = np.std([tree.feature_importances_ for tree in grid_tree_acc.best_estimator_.estimators_], axis=0)

sorted_idx = np.argsort(feature_importance)
pos = np.arange(sorted_idx.shape[0]) + .5
fig = plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.barh(pos, feature_importance[sorted_idx],xerr=std[sorted_idx][::-1], align='center')
plt.yticks(pos, np.array(list(X.columns))[sorted_idx])
plt.title('Feature Importance (MDI)',fontsize=10)

result = permutation_importance(RF, X_test, y_test, n_repeats=10,
                                random_state=42, n_jobs=2)
sorted_idx = result.importances_mean.argsort()
plt.subplot(1, 2, 2)
plt.boxplot(result.importances[sorted_idx].T,
            vert=False, labels=np.array(list(X.columns))[sorted_idx])
plt.title("Permutation Importance (test set)",fontsize=10)
fig.tight_layout()
plt.show()


## plotting the fit
plt.plot(y,RF.predict(X),'ro')
plt.xlabel('True values')
plt.ylabel('Predicted values')
plt.title(str(sc.stats.pearsonr(y,RF.predict(X))[0]))


Solution - BONUS - an even better splitting strategy

![RF](image/BlockedTimeSeriesSplit.png)

In [ ]:
# %load solutions/solution_03_02quat.py
## Even better splitting strategy


# we define our own splitter class
class BlockingTimeSeriesSplit():
    def __init__(self, n_splits):
        self.n_splits = n_splits
    
    def get_n_splits(self, X, y, groups):
        return self.n_splits
    
    def split(self, X, y=None, groups=None):
        n_samples = len(X)
        k_fold_size = n_samples // self.n_splits
        indices = np.arange(n_samples)

        margin = 0
        for i in range(self.n_splits):
            start = i * k_fold_size
            stop = start + k_fold_size
            mid = int(0.8 * (stop - start)) + start
            yield indices[start: mid], indices[mid + margin: stop]
            
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import TimeSeriesSplit
y = np.array(features['actual'])

# Remove the labels from the features
# axis 1 refers to the columns
X= features.drop(['Unnamed: 0', 'year', 'month', 'day',
       'actual', 'forecast_noaa', 'forecast_acc', 'forecast_under',
       'week_Fri', 'week_Mon', 'week_Sat', 'week_Sun', 'week_Thurs',
       'week_Tues', 'week_Wed'], axis = 1)


X_train=np.array(X)[:int(len(X.index)*0.75),:]                                                                           
X_test=np.array(X)[int(len(X.index)*0.75):,:]
y_train=np.array(y)[:int(len(X.index)*0.75)]
y_test=np.array(y)[int(len(X.index)*0.75):]
grid_values = {'criterion': ['squared_error'],
                'max_depth':[2,5,7],
               'min_samples_split':[4],
              'min_samples_leaf':[2]}
#with the range over which you want it to be tested.
tscv = BlockingTimeSeriesSplit(n_splits=5)


    
#Feed it to the GridSearchCV with the right
#score over which the decision should be taken    
grid_tree_acc = GridSearchCV(RandomForestRegressor(), 
                            param_grid = grid_values, 
                             scoring='r2',
                             cv=tscv, 
                             n_jobs=-1)

grid_tree_acc.fit(X_train, y_train)



print('Grid best parameter (max. r2): ', grid_tree_acc.best_params_)#get the best parameters
print('Grid best score (r2): ', grid_tree_acc.best_score_)#get the best score calculated from the train/validation
#dataset

y_decision_fn_scores_acc=grid_tree_acc.score(X_test,y_test)
print('Grid best parameter (max. r2) model on test: ', y_decision_fn_scores_acc)# get the equivalent score on the test
#dataset : again this is the important metric

## looking at feature importance 
RF = grid_tree_acc.best_estimator_
W=RF.feature_importances_#get the weights

sorted_features=sorted([[list(X.columns)[i],abs(W[i])] for i in range(len(W))],key=itemgetter(1),reverse=True)

print('Features sorted per importance in discriminative process')
print(sorted_features)


from sklearn.inspection import permutation_importance

feature_importance = RF.feature_importances_
std = np.std([tree.feature_importances_ for tree in grid_tree_acc.best_estimator_.estimators_], axis=0)

sorted_idx = np.argsort(feature_importance)
pos = np.arange(sorted_idx.shape[0]) + .5
fig = plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.barh(pos, feature_importance[sorted_idx],xerr=std[sorted_idx][::-1], align='center')
plt.yticks(pos, np.array(list(X.columns))[sorted_idx])
plt.title('Feature Importance (MDI)',fontsize=10)

result = permutation_importance(RF, X_test, y_test, n_repeats=10,
                                random_state=42, n_jobs=2)
sorted_idx = result.importances_mean.argsort()
plt.subplot(1, 2, 2)
plt.boxplot(result.importances[sorted_idx].T,
            vert=False, labels=np.array(list(X.columns))[sorted_idx])
plt.title("Permutation Importance (test set)",fontsize=10)
fig.tight_layout()
plt.show()


## plotting the fit
plt.plot(y,RF.predict(X),'ro')
plt.xlabel('True values')
plt.ylabel('Predicted values')
plt.title(str(sc.stats.pearsonr(y,RF.predict(X))[0]))

<br>

[Back to the ToC](#toc)

<br>

# Annexes <a id='annex'></a>

### Features selection

In [ ]:
df=sns.load_dataset("iris")
# Here we use the data loader from seaborn but such data loaders also exist with scikit-learn and are more generally delt
#with the dataframe handler pandas
df.head()

In [ ]:
sns.pairplot(df,hue="species")
# Seaborn allows you to 'split' your data according to a chosen parameter hue. Here I chose to color split the data according
#to the target
#description diagonal

What do you get from the plots above?

Looking at the diagonal of these plots : petal features separate the species more efficiently than sepal features.

There is a very strong correlation between `petal_length` and `petal_width` : those two features are probably so similar that keeping them both could be redundant.

The least correlation visible seems to be between `sepal_width` and all the others.

By itself `sepal_width` is not good at differentiating species but associated with other features we can already see groups forming by species. And since they are very much non-colinear I would say that, in dimension two, `petal_length` and `sepal_width` are already a good pick for low dimensions models.

You can actually quantify the correlation between features by calling the `corr()` function in pandas. You would prefer (and sometime is requiered) having a subset of features that are not correlated to each others.

In [ ]:
df_corr = df.corr()

sns.clustermap(df_corr,
               figsize=(8,8),
               z_score=None,
               row_cluster=True,
               col_cluster=True,
               method='ward',
               cmap='coolwarm',vmax=1,vmin=-1, 
               annot=True, annot_kws={"size": 13},cbar_kws={"label": 'Pearson\ncorrelation'})
## sns allows you to do a hierarchical clustering that simply
plt.show()

##### Classification
One thing (among others) that you can do is to look for a **subset of features that seems to be important to describe the target class**. It's like the pairplots above but instead of just looking at it you choose the features you want to keep.

You can choose different metrics for 'how important to describe the class' a feature is. 
Many of those metrics utilize concepts that we haven't introduced yet, in contexts that we haven't seen yet, so I will introduce two metrics for classification that don't need too much of *a priori* knowledge. 

`Scikit-learn` lets you specify a threshold on the features are kept, either as:
* a direct number: `SelectKBest`.
* important features from a percentile of your top importance score: `SelectPercentile`.
* an error type: `SelectFpr` or `SelectFdr` (see course 2 logistic regression part).


`Scikit-learn` offers you different scores to calculate the importance of your features.

* **ANOVA-F** : F=$\frac{Var_{feature\_i}(Between\_class)}{Var_{feature\_i}(Within\_class)}$. 

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
**F** itself gives you how much a feature $i$ variance is different between classes, normalized by the intrinsic variance of that feature per class. 

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; 
So if **F** is big it means that the variation that you observe between classes is big compared to the variance of this feature : 

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; 
it behaves differently for different classes so it it is a good feature to keep for the classification. 

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; 
To this **F** is associated a **p-value** that you would use for scoring.


* **Chi2** ($\chi^{2}$) test. 

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
You suppose the null hypothesis that this feature $i$ is homogenously distributed among classes

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
and so you are expecting that its representation in different classes should be very similar to what you can calculate for the bulk data

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
 i.e. $\frac{\Sigma^{n\_points} feature_{i}}{n\_points}$.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
You then compare the actual distribution of this feature in different classes to your null model predictions. If this **sum of square differences**: 

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
$\Sigma^{n\_class}_{k}\frac{(expected\_form\_null\_hypothesis_{k}-observed_{k})^{2}}{observed}$

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; 
is big then the null hypothesis has to be rejected and this feature is significant for classifying. 

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
The sum of these square quantities over the different classes asymptotically follows a $\chi^{2}$ 

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
distribution and thus you have access to a **p-value for scoring**.


Another score would be to use the amount of [Mutual Information](https://en.wikipedia.org/wiki/Mutual_information) shared between a feature and our target. 

The way this mutual information is caclulated is out of the scope of this class as it is a bit technical.

##### For regression just use correlation or Mutual Iformation

In [ ]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

skb = SelectKBest(chi2, k=2)#creating the object SelectKBest and settling for 2 best features (k=2) in term of chi2 score
skb.fit(df[list(df.columns)[:-1]], df[list(df.columns)[-1]])#calculating the chi2 for each features

dico_pval={df.columns[i]:v for i,v in enumerate(skb.pvalues_)}
print("features Chi2 scores (p-values):")#all the features and the chi2 pvalues associated. use .pvalues_
for feature,pval in dico_pval.items() :
    print('\t',feature , ':' , pval )

X_new=skb.transform(df[list(df.columns)[:-1]])# keep only the k=2 best features according to the score

print("New data with only the k=2 best features kept :")
print(X_new[:5,]) #printing only the 5 first entries
print('...')

### Multi classes

In [ ]:
X3, y3 = make_blobs(n_samples=120, centers=3,cluster_std=3, random_state=6)# 120 points, 3 blobs/clusters with some spread=3
#Random_state is here just to be sure that every time you will get the same blobs. If you change the random_state or do not
#specify it then you will get a new plot every time you call the function (random seed)

Of course all of that can be applied to a multi-classes classification. How is it tipically done?

There are many different ways of tackling the problem, that end up being a combination of these 4 elements :

- **Either you treat the problem as one class vs one class**.

- **Or you treat the problem as a one class vs the rest : you subdivide the problem into three different problems either your are class 1 and you consider the other classes as being one big class "non 1", and you do the same for the other class**.
- **You change your loss function to a multinomial one : softmax intead of a sigmoid.** 

In any case you need to decide **how you are going to agglomerate those different statistics (different ROC curves for example)**:

- **micro average** : pull all raw numbers together (eg. number of FP, TP), group them and then calculate your overall statistic (eg. TPR)
- **macro average** : calculate each statistics separately and then do the average.

Think about the differences induced by those metrics. Why should you use one more than the other? Or maybe you should always use all of them?

Spoiler it has to do with overall separability and balance between the different class.


What strategy your logistic regression uses so you can plot the right curves, is a tricky question. For a first pass on your data always set the multiclasses method to be ovr (one vs rest) : understanding the hyperplanes relation to decision probability and the ROC curve is more intuitive that way, and I believe less sensitive to imbalance dataset.

In [ ]:
from utils import contour_lr_more 
#one vs rest implementation
contour_lr_more('l2',X3,y3,10,'ovr')

In [ ]:
from utils import contour_lr_more 
contour_lr_more('l2',X3,y3,10,'multinomial')
